<a href="https://colab.research.google.com/github/roshancharlie/College-Chatbot-Using-ML-and-NLP/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing The Neccesary Libraries

In [2]:
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')

# Creating Query-Based Intents for the Dataset

### Downloading NLP Package

In [23]:
intents=[
    {
        "tag": "greeting",
        "patterns": [
            "Hi",
            "Hello",
            "Hey",
            "How are you",
            "What's up"
        ],
        "responses": [
            "Hi there",
            "Hello",
            "Hey",
            "I'm fine, thank you",
            "Nothing much"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": [
            "Bye",
            "See you later",
            "Goodbye",
            "Take care"
        ],
        "responses": [
            "Goodbye",
            "See you later",
            "Take care"
        ]
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thank you",
            "Thanks",
            "Thanks a lot",
            "I appreciate it"
        ],
        "responses": [
            "You're welcome",
            "No problem",
            "Glad I could help"
        ]
    },
    {
        "tag": "about",
        "patterns": [
            "What can you do",
            "Who are you",
            "What are you",
            "What is your purpose"
        ],
        "responses": [
            "I am a chatbot",
            "My purpose is to assist you",
            "I can answer questions and provide assistance"
        ]
    },
    {
        "tag": "help",
        "patterns": [
            "Help",
            "I need help",
            "Can you help me",
            "What should I do"
        ],
        "responses": [
            "Sure, what do you need help with?",
            "I'm here to help. What's the problem?",
            "How can I assist you?"
        ]
    },
    {
        "tag": "age",
        "patterns": [
            "How old are you",
            "What's your age"
        ],
        "responses": [
            "I don't have an age. I'm a chatbot.",
            "I was just born in the digital world.",
            "Age is just a number for me."
        ]
    },
    {
        "tag": "weather",
        "patterns": [
            "What's the weather like",
            "How's the weather today"
        ],
        "responses": [
            "I'm sorry, I cannot provide real-time weather information.",
            "You can check the weather on a weather app or website."
        ]
    },
    {
        "tag": "budget",
        "patterns": [
            "How can I make a budget",
            "What's a good budgeting strategy",
            "How do I create a budget"
        ],
        "responses": [
            "To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.",
            "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.",
            "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."
        ]
    },
    {
        "tag": "credit_score",
        "patterns": [
            "What is a credit score",
            "How do I check my credit score",
            "How can I improve my credit score"
        ],
        "responses": [
            "A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.",
            "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."
        ]
    },
    {
        "tag": "name",
        "patterns": [
            "What's your name",
            "Do you have a name",
            "What should I call you"
        ],
        "responses": [
            "You can call me Chatbot.",
            "My name is Chatbot."
        ]
    },
    {
        "tag": "favorite_color",
        "patterns": [
            "What's your favorite color",
            "Do you like any color"
        ],
        "responses": [
            "I'm a chatbot, so I don't have a favorite color.",
            "As an AI, I don't have preferences like humans do."
        ]
    },
    {
        "tag": "hobby",
        "patterns": [
            "What do you do for fun",
            "Do you have any hobbies"
        ],
        "responses": [
            "I don't have hobbies as I'm here to assist you.",
            "I'm always ready to chat with you, so that's my favorite thing to do!"
        ]
    },
    {
        "tag": "time",
        "patterns": [
            "What's the time",
            "Can you tell me the time"
        ],
        "responses": [
            "I'm sorry, but I don't have access to real-time information like the current time.",
            "You can check the time on your device or use a clock."
        ]
    },
    {
        "tag": "joke",
        "patterns": [
            "Tell me a joke",
            "Do you know any jokes",
            "Make me laugh"
        ],
        "responses": [
            "Why don’t scientists trust atoms? Because they make up everything!",
            "Why don't skeletons fight each other? They don't have the guts!",
            "Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them!"
        ]
    },
    {
        "tag": "food",
        "patterns": [
            "What's your favorite food",
            "Do you like to eat",
            "Tell me about food"
        ],
        "responses": [
            "As a chatbot, I don't eat, but I'm here to help with any questions you have about food!",
            "I may not have taste buds, but I can assist you in finding recipes or restaurants!"
        ]
    },
    {
        "tag": "movies",
        "patterns": [
            "What's your favorite movie",
            "Recommend me a movie",
            "Tell me about movies"
        ],
        "responses": [
            "As an AI, I don't watch movies, but I can suggest some popular ones like The Shawshank Redemption, Inception, or The Godfather.",
            "I may not have personal preferences, but I can help you find movies based on your taste!"
        ]
    },
    {
        "tag": "technology",
        "patterns": [
            "What's the latest tech news",
            "Tell me about technology",
            "What's new in tech"
        ],
        "responses": [
            "As an AI language model, I don't have real-time data, but you can check technology news websites for the latest updates.",
            "Technology is constantly evolving. Stay updated with tech blogs and news sites to know about the latest advancements!"
        ]
    },
    {
        "tag": "compliment",
        "patterns": [
            "You're awesome",
            "You're great",
            "I like you"
        ],
        "responses": [
            "Thank you! I'm just a program, but I'm here to assist and make your experience better.",
            "I'm glad you think so! My purpose is to help and provide useful information."
        ]
    },
    {
        "tag": "meaning_of_life",
        "patterns": [
            "What is the meaning of life",
            "Why are we here",
            "What's the purpose of life"
        ],
        "responses": [
            "The meaning of life is a philosophical question. Different people and cultures have different beliefs about it.",
            "The purpose of life is subjective and can vary from person to person. Some find purpose in relationships, careers, or helping others."
        ]
    },
    {
        "tag": "sports",
        "patterns": [
            "Tell me about sports",
            "What's your favorite sport",
            "Any sports news"
        ],
        "responses": [
            "While I don't have personal preferences, sports can be exciting! Keep an eye on sports news websites for the latest updates and scores.",
            "Sports are a great way to stay active and entertained. There's a wide range of sports to explore and enjoy!"
        ]
    },
    {
        "tag": "pets",
        "patterns": [
            "Do you have any pets",
            "Tell me about pets",
            "Do you like animals"
        ],
        "responses": [
            "As an AI, I don't have pets, but I'm interested in animals and can answer your questions about them.",
            "Pets can bring joy and companionship into our lives. Many people love keeping dogs, cats, birds, and more as their pets."
        ]
    },
    {
        "tag": "travel",
        "patterns": [
            "Where is a good place to travel",
            "Recommend me a travel destination",
            "Tell me about your favorite travel spot"
        ],
        "responses": [
            "I'm just a chatbot, so I don't travel, but there are many beautiful destinations around the world to explore. It depends on your preferences, such as beaches, mountains, historical sites, or bustling cities.",
            "Traveling allows you to experience different cultures, try new cuisines, and create lasting memories."
        ]
    },
    {
        "tag": "books",
        "patterns": [
            "What's your favorite book",
            "Recommend me a book to read",
            "Tell me about books"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are countless amazing books in various genres. Some popular ones include Harry Potter, To Kill a Mockingbird, and 1984.",
            "Reading books can broaden your knowledge, enhance creativity, and provide a great way to relax."
        ]
    },
    {
        "tag": "education",
        "patterns": [
            "Tell me about education",
            "What's the importance of education",
            "How to study effectively"
        ],
        "responses": [
            "Education is the process of acquiring knowledge, skills, values, and attitudes. It plays a crucial role in personal and societal development.",
            "Studying effectively involves setting clear goals, creating a conducive study environment, staying organized, taking regular breaks, and seeking help when needed."
        ]
    },
    {
        "tag": "health",
        "patterns": [
            "How to stay healthy",
            "Tell me about health tips",
            "What's the importance of fitness"
        ],
        "responses": [
            "To stay healthy, maintain a balanced diet, engage in regular physical activity, get enough sleep, manage stress, and avoid harmful habits.",
            "Fitness is essential for overall well-being and can improve your mood, energy levels, and longevity."
        ]
    },
    {
        "tag": "coding",
        "patterns": [
            "Tell me about coding",
            "How to start coding",
            "What's the best programming language"
        ],
        "responses": [
            "Coding is the process of writing instructions for computers to execute tasks. It powers software, websites, and apps.",
            "If you want to start coding, choose a programming language like Python, Java, or JavaScript, and explore online tutorials and courses."
        ]
    },
    {
        "tag": "art",
        "patterns": [
            "Tell me about art",
            "What's your favorite artwork",
            "How to appreciate art"
        ],
        "responses": [
            "Art comes in various forms, such as paintings, sculptures, music, literature, and more. It's a way to express emotions and ideas.",
            "Appreciating art involves being open-minded, observing details, understanding the context, and exploring different art movements."
        ]
    },
    {
        "tag": "career",
        "patterns": [
            "How to find a job",
            "Tell me about career development",
            "What's the best career advice"
        ],
        "responses": [
            "To find a job, identify your skills and interests, create a compelling resume, network, and apply to suitable positions.",
            "Career development involves setting goals, continuous learning, seeking mentorship, and adapting to the evolving job market."
        ]
    },
    {
        "tag": "technology_help",
        "patterns": [
            "How can you assist with technology",
            "Tell me about tech support",
            "Can you help with computer issues"
        ],
        "responses": [
            "As an AI chatbot, I can provide information and basic troubleshooting for common technology problems.",
            "For complex technical issues, it's best to consult a qualified IT professional."
        ]
    },
    {
        "tag": "history",
        "patterns": [
            "Tell me about history",
            "What's your favorite historical period",
            "How can I learn about historical events"
        ],
        "responses": [
            "History is the study of past events, societies, and civilizations. It provides insights into our roots and informs our future.",
            "Learning about history can involve reading books, visiting museums, watching documentaries, and attending history lectures."
        ]
    },
    {
        "tag": "music",
        "patterns": [
            "What's your favorite music",
            "Recommend me a song",
            "Tell me about music genres"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are various music genres like pop, rock, classical, hip-hop, and more.",
            "Music can evoke emotions, improve mood, and be a source of inspiration."
        ]
    },
    {
        "tag": "exercise",
        "patterns": [
            "How to stay fit",
            "Tell me about exercise",
            "What's the importance of physical activity"
        ],
        "responses": [
            "Staying fit involves a mix of cardiovascular exercises, strength training, flexibility exercises, and a balanced diet.",
            "Regular exercise can enhance physical health, mental well-being, and boost energy levels."
        ]
    },
    {
        "tag": "mindfulness",
        "patterns": [
            "What is mindfulness",
            "Tell me about mindfulness techniques",
            "How to practice mindfulness"
        ],
        "responses": [
            "Mindfulness is the practice of being fully present in the moment, observing thoughts and feelings without judgment.",
            "Mindfulness techniques include meditation, deep breathing, body scanning, and mindful eating."
        ]
    },
    {
        "tag": "science",
        "patterns": [
            "Tell me about science",
            "What's your favorite branch of science",
            "How does science impact our lives"
        ],
        "responses": [
            "Science is the systematic study of the natural world, aiming to understand phenomena and make discoveries.",
            "Science has led to advancements in medicine, technology, agriculture, and our understanding of the universe."
        ]
    },
    {
        "tag": "gaming",
        "patterns": [
            "Do you play games",
            "Tell me about gaming",
            "What's your favorite video game"
        ],
        "responses": [
            "As an AI, I don't play games, but I can help you with information about various video games and gaming platforms.",
            "Gaming is a popular form of entertainment enjoyed by people of all ages."
        ]
    },
    {
        "tag": "positivity",
        "patterns": [
            "Spread some positivity",
            "Tell me a positive quote",
            "How to stay positive"
        ],
        "responses": [
            "Every day is a new opportunity to make a positive impact. You are capable of great things!",
            "Remember, you are strong, resilient, and capable of overcoming challenges."
        ]
    },
    {
        "tag": "cooking",
        "patterns": [
            "Tell me about cooking",
            "What's your favorite dish",
            "Cooking tips for beginners"
        ],
        "responses": [
            "Cooking is the art of preparing food, and it allows you to experiment with flavors and create delicious meals.",
            "For beginners, start with simple recipes, use fresh ingredients, and don't be afraid to try new techniques."
        ]
    },
    {
        "tag": "relationship",
        "patterns": [
            "How to maintain a healthy relationship",
            "Tell me about love",
            "Relationship advice"
        ],
        "responses": [
            "Communication, trust, and mutual respect are essential for a healthy and fulfilling relationship.",
            "Love is a complex and beautiful emotion that connects people on a deep level."
        ]
    },
    {
        "tag": "nature",
        "patterns": [
            "Tell me about nature",
            "What's your favorite natural wonder",
            "How to protect the environment"
        ],
        "responses": [
            "Nature encompasses the beauty and diversity of the world, including landscapes, wildlife, and ecosystems.",
            "Protecting the environment involves reducing waste, conserving resources, and supporting sustainable practices."
        ]
    },
    {
        "tag": "productivity",
        "patterns": [
            "How to be more productive",
            "Tell me about time management",
            "Productivity tips"
        ],
        "responses": [
            "To be more productive, prioritize tasks, set goals, eliminate distractions, and take regular breaks.",
            "Effective time management can lead to increased efficiency and reduced stress."
        ]
    },
    {
        "tag": "travel_tips",
        "patterns": [
            "Tell me about travel tips",
            "How to pack for a trip",
            "What to consider while traveling"
        ],
        "responses": [
            "Travel tips include packing light, carrying essential documents, researching your destination, and respecting local customs.",
            "While traveling, be open to new experiences, try local cuisines, and be mindful of your surroundings."
        ]
    },
    {
        "tag": "languages",
        "patterns": [
            "Tell me about languages",
            "How to learn a new language",
            "What's the importance of multilingualism"
        ],
        "responses": [
            "Languages are a crucial aspect of human communication and culture, with thousands of languages spoken worldwide.",
            "Learning a new language can expand your horizons, enhance cognitive abilities, and facilitate cross-cultural interactions."
        ]
    },
    {
        "tag": "inspiration",
        "patterns": [
            "I need some inspiration",
            "Tell me an inspiring story",
            "How to stay motivated"
        ],
        "responses": [
            "You are capable of achieving remarkable things. Believe in yourself and never give up!",
            "The journey to success may have challenges, but it's the persistence and determination that lead to extraordinary accomplishments."
        ]
    },
    {
        "tag": "finance_tips",
        "patterns": [
            "Tell me about finance tips",
            "How to save money",
            "Investment advice"
        ],
        "responses": [
            "Finance tips include budgeting, saving a portion of your income, avoiding unnecessary expenses, and investing wisely for the future.",
            "Investing in stocks, mutual funds, or real estate can help grow your wealth over time."
        ]
    },
    {
        "tag": "artificial_intelligence",
        "patterns": [
            "Tell me about artificial intelligence",
            "How does AI work",
            "AI applications"
        ],
        "responses": [
            "Artificial intelligence is the simulation of human intelligence in machines, enabling them to learn and perform tasks.",
            "AI has diverse applications, including voice assistants, self-driving cars, recommendation systems, and medical diagnostics."
        ]
    },
    {
        "tag": "motivation",
        "patterns": [
            "I need some motivation",
            "Tell me an encouraging quote",
            "How to overcome obstacles"
        ],
        "responses": [
            "Believe in yourself. You have the strength to overcome any obstacle and achieve your dreams!",
            "Obstacles are stepping stones to success. Embrace challenges as opportunities for growth and learning."
        ]
    },
    {
        "tag": "future",
        "patterns": [
            "Tell me about the future",
            "What will the future look like",
            "Future technology"
        ],
        "responses": [
            "The future is uncertain, but advancements in technology, medicine, and space exploration hold great potential.",
            "The future of technology may include AI advancements, renewable energy solutions, and even the possibility of space colonization."
        ]
    },
    {
        "tag": "movies_series",
        "patterns": [
            "Tell me about movies or series",
            "Recommend me a TV show",
            "Movie genres"
        ],
        "responses": [
            "Movies and TV series offer a wide range of genres, including drama, comedy, sci-fi, fantasy, thriller, and more.",
            "Binge-watching your favorite series can be a great way to relax and unwind."
        ]
    },
    {
        "tag": "self_improvement",
        "patterns": [
            "How to improve myself",
            "Tell me about personal growth",
            "Tips for self-development"
        ],
        "responses": [
            "Self-improvement involves setting goals, embracing continuous learning, staying positive, and practicing self-care.",
            "Embrace challenges as opportunities for growth, and focus on becoming the best version of yourself."
        ]
    },
    {
        "tag": "robotics",
        "patterns": [
            "Tell me about robotics",
            "How do robots work",
            "Future of robotics"
        ],
        "responses": [
            "Robotics is the study and development of robots, which are machines capable of performing tasks autonomously or semi-autonomously.",
            "The future of robotics may include advancements in AI, human-robot collaboration, and robotics in industries like healthcare and manufacturing."
        ]
    },
    {
        "tag": "philosophy",
        "patterns": [
            "Tell me about philosophy",
            "What's the purpose of life",
            "Philosophical questions"
        ],
        "responses": [
            "Philosophy is the study of fundamental questions about existence, knowledge, values, reason, mind, and language.",
            "Philosophical questions explore the nature of reality, ethics, free will, consciousness, and the meaning of life."
        ]
    },
    {
        "tag": "coding_languages",
        "patterns": [
            "Tell me about coding languages",
            "Which programming language to learn",
            "Popular coding languages"
        ],
        "responses": [
            "Coding languages include Python, Java, JavaScript, C++, and more. The choice depends on your interests and the application you want to develop.",
            "Python is often recommended for beginners due to its readability and versatility."
        ]
    },
    {
        "tag": "virtual_reality",
        "patterns": [
            "Tell me about virtual reality",
            "How does VR work",
            "Applications of virtual reality"
        ],
        "responses": [
            "Virtual reality is a computer-generated simulation that allows users to interact with a 3D environment.",
            "VR has applications in gaming, training, education, architecture, and therapeutic interventions."
        ]
    },
    {
        "tag": "space_exploration",
        "patterns": [
            "Tell me about space exploration",
            "What's happening in space",
            "Mars colonization"
        ],
        "responses": [
            "Space exploration involves sending robotic missions and astronauts to study celestial bodies and understand the universe.",
            "Mars colonization is a long-term goal, and scientists are conducting research and planning potential missions."
        ]
    },
    {
        "tag": "emotional_intelligence",
        "patterns": [
            "Tell me about emotional intelligence",
            "Why is EQ important",
            "How to develop emotional intelligence"
        ],
        "responses": [
            "Emotional intelligence (EQ) is the ability to understand and manage emotions, both in oneself and others.",
            "Developing emotional intelligence involves self-awareness, empathy, effective communication, and emotional regulation."
        ]
    },
    {
        "tag": "cybersecurity",
        "patterns": [
            "Tell me about cybersecurity",
            "How to protect against cyber threats",
            "Cybersecurity best practices"
        ],
        "responses": [
            "Cybersecurity involves protecting computers, networks, and data from unauthorized access, attacks, and damage.",
            "Best practices include using strong passwords, keeping software updated, avoiding suspicious links, and using antivirus software."
        ]
    },
    {
        "tag": "creativity",
        "patterns": [
            "Tell me about creativity",
            "How to boost creativity",
            "Importance of creativity"
        ],
        "responses": [
            "Creativity is the ability to think outside the box, generate new ideas, and solve problems innovatively.",
            "To boost creativity, engage in activities like writing, drawing, brainstorming, or trying new experiences."
        ]
    },
    {
        "tag": "futuristic_technology",
        "patterns": [
            "Tell me about futuristic technology",
            "What will technology be like in 50 years",
            "Next-gen innovations"
        ],
        "responses": [
            "Futuristic technology may include advancements in AI, nanotechnology, biotechnology, quantum computing, and space exploration.",
            "Predicting the exact state of technology in the future is challenging, but progress is expected to be exponential."
        ]
    },
    {
        "tag": "entrepreneurship",
        "patterns": [
            "Tell me about entrepreneurship",
            "How to start a business",
            "Keys to entrepreneurial success"
        ],
        "responses": [
            "Entrepreneurship involves identifying opportunities, taking risks, and creating new ventures.",
            "Success in entrepreneurship requires determination, adaptability, effective leadership, and a problem-solving mindset."
        ]
    },
    {
        "tag": "internet_of_things",
        "patterns": [
            "Tell me about the Internet of Things (IoT)",
            "How does IoT work",
            "Applications of IoT"
        ],
        "responses": [
            "The Internet of Things refers to the network of physical objects embedded with sensors, software, and connectivity.",
            "IoT has applications in smart homes, healthcare, agriculture, transportation, and industrial automation."
        ]
    },
    {
        "tag": "universe",
        "patterns": [
            "Tell me about the universe",
            "How big is the universe",
            "Astrology vs. astronomy"
        ],
        "responses": [
            "The universe is vast, containing billions of galaxies, each with billions of stars.",
            "Astrology is the study of celestial bodies' influence on human affairs, while astronomy is the scientific study of celestial objects and phenomena."
        ]
    },
    {
        "tag": "social_media",
        "patterns": [
            "Tell me about social media",
            "Pros and cons of social networking",
            "How to use social media responsibly"
        ],
        "responses": [
            "Social media platforms allow people to connect, share content, and stay updated on news and events.",
            "Using social media responsibly involves protecting privacy, avoiding misinformation, and fostering positive interactions."
        ]
    },
    {
        "tag": "cuisine",
        "patterns": [
            "Tell me about different cuisines",
            "What's your favorite dish",
            "Traditional foods from different cultures"
        ],
        "responses": [
            "Cuisine varies across regions and cultures, offering a diverse range of flavors and ingredients.",
            "Trying foods from different cultures can be a delightful culinary adventure."
        ]
    },
    {
        "tag": "happiness",
        "patterns": [
            "Tell me about happiness",
            "How to find happiness",
            "Keys to a happy life"
        ],
        "responses": [
            "Happiness is a subjective and individual experience, and it can be found in meaningful relationships, personal achievements, and gratitude.",
            "Focusing on positive aspects, practicing mindfulness, and helping others contribute to a happier life."
        ]
    },
    {
        "tag": "self_care",
        "patterns": [
            "Tell me about self-care",
            "How to practice self-care",
            "Importance of self-love"
        ],
        "responses": [
            "Self-care involves taking care of one's physical, emotional, and mental well-being.",
            "Prioritizing self-care can lead to improved overall health and a more balanced lifestyle."
        ]
    },
    {
        "tag": "augmented_reality",
        "patterns": [
            "Tell me about augmented reality",
            "How does AR work",
            "AR applications"
        ],
        "responses": [
            "Augmented reality overlays digital content onto the real world, enhancing the user's experience.",
            "AR has applications in gaming, education, retail, and industrial training."
        ]
    },
    {
        "tag": "global_warming",
        "patterns": [
            "Tell me about global warming",
            "Climate change facts",
            "How to reduce carbon footprint"
        ],
        "responses": [
            "Global warming refers to the long-term increase in Earth's average temperature due to human activities, primarily the burning of fossil fuels.",
            "Reducing carbon footprint involves energy conservation, using renewable energy sources, and supporting eco-friendly practices."
        ]
    },
    {
        "tag": "data_privacy",
        "patterns": [
            "Tell me about data privacy",
            "Why is data security important",
            "Protecting personal information online"
        ],
        "responses": [
            "Data privacy refers to safeguarding personal and sensitive information from unauthorized access and misuse.",
            "Protecting personal information online involves using strong passwords, enabling two-factor authentication, and being cautious about sharing sensitive data."
        ]
    },
    {
        "tag": "positivity_quotes",
        "patterns": [
            "Tell me a positive quote",
            "Inspirational quotes",
            "Motivational sayings"
        ],
        "responses": [
            "The future belongs to those who believe in the beauty of their dreams. - Eleanor Roosevelt",
            "In the middle of every difficulty lies opportunity. - Albert Einstein",
            "Your time is limited, don't waste it living someone else's life. - Steve Jobs"
        ]
    },
    {
        "tag": "virtual_assistant",
        "patterns": [
            "Tell me about virtual assistants",
            "How does a virtual assistant work",
            "Popular virtual assistant applications"
        ],
        "responses": [
            "Virtual assistants are AI-powered programs that perform tasks and provide information through natural language interaction.",
            "Popular virtual assistants include Siri, Alexa, Google Assistant, and Cortana."
        ]
    },
    {
        "tag": "emerging_technologies",
        "patterns": [
            "Tell me about emerging technologies",
            "What's the latest in tech",
            "Next big tech innovations"
        ],
        "responses": [
            "Emerging technologies include 5G, blockchain, quantum computing, biotechnology, and renewable energy solutions.",
            "Stay updated with tech news to learn about the latest breakthroughs and innovations."
        ]
    },
    {
        "tag": "philanthropy",
        "patterns": [
            "Tell me about philanthropy",
            "Importance of giving back",
            "How to make a positive impact"
        ],
        "responses": [
            "Philanthropy involves charitable acts and contributions to promote the welfare of others and support important causes.",
            "Giving back to the community can create a positive ripple effect and make a meaningful difference in people's lives."
        ]
    },
    {
        "tag": "sustainability",
        "patterns": [
            "Tell me about sustainability",
            "Why is sustainable living important",
            "How to be more eco-friendly"
        ],
        "responses": [
            "Sustainability aims to meet current needs without compromising the ability of future generations to meet their needs.",
            "Adopting eco-friendly practices, reducing waste, and supporting sustainable products contribute to a greener planet."
        ]
    },
    {
        "tag": "gene_editing",
        "patterns": [
            "Tell me about gene editing",
            "How does gene editing work",
            "Ethical considerations of genetic engineering"
        ],
        "responses": [
            "Gene editing allows scientists to modify DNA, potentially treating genetic diseases and enhancing desirable traits.",
            "Gene editing raises ethical questions about responsible use, unintended consequences, and potential misuse."
        ]
    },
    {
        "tag": "machine_learning",
        "patterns": [
            "Tell me about machine learning",
            "How does ML work",
            "Applications of machine learning"
        ],
        "responses": [
            "Machine learning is a subset of AI that enables systems to learn from data and improve performance over time.",
            "ML has applications in image recognition, natural language processing, recommendation systems, and predictive analytics."
        ]
    },
    {
        "tag": "gamification",
        "patterns": [
            "Tell me about gamification",
            "How does gamification work",
            "Benefits of gamified learning"
        ],
        "responses": [
            "Gamification involves applying game elements, such as points and rewards, to non-game contexts to engage and motivate users.",
            "Gamified learning can enhance engagement, foster competition, and make learning more enjoyable."
        ]
    },
    {
        "tag": "brain_computer_interface",
        "patterns": [
            "Tell me about brain-computer interfaces",
            "How does BCI work",
            "BCI applications"
        ],
        "responses": [
            "Brain-computer interfaces establish direct communication between the brain and external devices, enabling control and data exchange.",
            "BCIs have applications in healthcare, assistive technology, and potential future applications in communication and entertainment."
        ]
    },
    {
        "tag": "depression",
        "patterns": [
            "Tell me about depression",
            "Signs of depression",
            "How to support someone with depression"
        ],
        "responses": [
            "Depression is a mental health disorder characterized by persistent sadness, loss of interest, and changes in behavior and mood.",
            "Supporting someone with depression involves offering empathy, listening without judgment, and encouraging professional help."
        ]
    },
    {
        "tag": "nanotechnology",
        "patterns": [
            "Tell me about nanotechnology",
            "How does nanotech work",
            "Nanotech applications"
        ],
        "responses": [
            "Nanotechnology involves manipulating materials and structures at the nanoscale, often on the atomic or molecular level.",
            "Nanotechnology has applications in medicine, electronics, energy, and environmental remediation."
        ]
    },
    {
        "tag": "artificial_superintelligence",
        "patterns": [
            "Tell me about artificial superintelligence",
            "What is ASI",
            "Potential impacts of superintelligent AI"
        ],
        "responses": [
            "Artificial Superintelligence (ASI) refers to highly autonomous AI systems that surpass human intelligence in all aspects.",
            "The potential impacts of ASI are a topic of debate, involving considerations of safety, ethics, and control."
        ]
    },
    {
        "tag": "virtual_currency",
        "patterns": [
            "Tell me about virtual currencies",
            "What is cryptocurrency",
            "Advantages of blockchain technology"
        ],
        "responses": [
            "Virtual currencies like Bitcoin are decentralized digital assets that operate on blockchain technology.",
            "Blockchain offers benefits such as transparency, security, reduced intermediaries, and potential applications beyond finance."
        ]
    },
    {
        "tag": "biomedical_engineering",
        "patterns": [
            "Tell me about biomedical engineering",
            "How does biomedical engineering work",
            "Medical innovations"
        ],
        "responses": [
            "Biomedical engineering applies principles of engineering and technology to solve medical and healthcare challenges.",
            "Medical innovations in biomedical engineering include prosthetics, medical imaging, regenerative medicine, and medical devices."
        ]
    },
    {
        "tag": "quantum_entanglement",
        "patterns": [
            "Tell me about quantum entanglement",
            "How does entanglement work",
            "Applications of quantum entanglement"
        ],
        "responses": [
            "Quantum entanglement is a quantum phenomenon where two or more particles become connected and behave as one, even when separated by great distances.",
            "Applications of quantum entanglement include quantum computing, quantum teleportation, and quantum communication."
        ]
    },
    {
        "tag": "books",
        "patterns": [
            "Recommend me a book",
            "Tell me about your favorite book",
            "Book genres"
        ],
        "responses": [
            "Reading is a great way to expand your knowledge and imagination. Some popular genres include fiction, non-fiction, mystery, fantasy, and romance.",
            "Books can transport you to different worlds, teach valuable lessons, and inspire personal growth."
        ]
    },
    {
        "tag": "movies",
        "patterns": [
            "Recommend me a movie",
            "Tell me about your favorite movie",
            "Movie genres"
        ],
        "responses": [
            "Movies offer a wide variety of genres, such as action, comedy, drama, thriller, sci-fi, and horror.",
            "Watching movies can be a fun and entertaining way to relax and unwind."
        ]
    },
    {
        "tag": "workout",
        "patterns": [
            "Tell me about different workout routines",
            "How to stay motivated for workouts",
            "Benefits of regular exercise"
        ],
        "responses": [
            "Workout routines can include cardio, strength training, yoga, and HIIT workouts. Find one that suits your fitness goals and preferences.",
            "Staying motivated for workouts can be easier when you set achievable goals, track progress, and find a workout buddy for accountability."
        ]
    },
    {
        "tag": "dancing",
        "patterns": [
            "Tell me about different dance styles",
            "How to start learning dancing",
            "Benefits of dancing"
        ],
        "responses": [
            "Dance styles vary from hip-hop, ballet, salsa, to contemporary and more. Choose a style that resonates with your interests.",
            "Dancing is not only a great form of exercise but also improves coordination, boosts mood, and promotes self-expression."
        ]
    },
    {
        "tag": "cooking_tips",
        "patterns": [
            "Tell me about kitchen hacks",
            "How to make cooking easier",
            "Quick and easy recipes"
        ],
        "responses": [
            "Kitchen hacks can save time and effort, like using lemon to prevent browning of fruits or using an ice cream scoop for cookie dough.",
            "For easy recipes, try one-pan meals, salads, or stir-fries that require minimal preparation and cooking time."
        ]
    },
    {
        "tag": "travel_destinations",
        "patterns": [
            "Tell me about popular travel destinations",
            "Where to travel next",
            "Unexplored travel spots"
        ],
        "responses": [
            "Popular travel destinations include Paris, Tokyo, New York, and Bali. Unexplored spots can be found in offbeat regions and lesser-known towns.",
            "Traveling allows you to experience new cultures, cuisines, and create lasting memories."
        ]
    },
    {
        "tag": "meditation",
        "patterns": [
            "Tell me about different meditation techniques",
            "How to meditate effectively",
            "Benefits of meditation"
        ],
        "responses": [
            "Meditation techniques include mindfulness, breathing exercises, loving-kindness meditation, and guided meditation.",
            "Meditation can reduce stress, improve focus, and promote emotional well-being."
        ]
    },
    {
        "tag": "home_decor",
        "patterns": [
            "Tell me about home decor ideas",
            "How to decorate a small space",
            "DIY home projects"
        ],
        "responses": [
            "Home decor ideas can involve adding plants, changing wall colors, and incorporating cozy textiles for a welcoming space.",
            "For small spaces, use multipurpose furniture, mirrors to create an illusion of space, and keep clutter to a minimum."
        ]
    },
    {
        "tag": "pet_care",
        "patterns": [
            "Tell me about pet care",
            "How to train a puppy",
            "Best pet-friendly activities"
        ],
        "responses": [
            "Pet care involves regular feeding, exercise, grooming, and regular vet check-ups to ensure your pet's well-being.",
            "To train a puppy, use positive reinforcement, consistency, and patience to establish good behaviors."
        ]
    },
    {
        "tag": "gardening",
        "patterns": [
            "Tell me about gardening tips",
            "How to start a vegetable garden",
            "Benefits of indoor plants"
        ],
        "responses": [
            "Gardening tips include choosing the right plants for your climate, watering properly, and using organic fertilizers.",
            "Growing a vegetable garden allows you to enjoy fresh produce and connect with nature."
        ]
    },
    {
        "tag": "parenting",
        "patterns": [
            "Tell me about parenting tips",
            "How to manage work-life balance as a parent",
            "Building a strong bond with children"
        ],
        "responses": [
            "Parenting tips involve active listening, setting boundaries, and fostering open communication with your children.",
            "Balancing work and family life can be achieved by prioritizing tasks, setting realistic expectations, and seeking support when needed."
        ]
    },
    {
        "tag": "time_management",
        "patterns": [
            "Tell me about time management techniques",
            "How to stay productive",
            "Avoiding procrastination"
        ],
        "responses": [
            "Time management techniques include creating to-do lists, using time blocks, and setting specific goals.",
            "To stay productive, identify your most productive hours, eliminate distractions, and take regular breaks."
        ]
    },
    {
        "tag": "online_learning",
        "patterns": [
            "Tell me about online learning platforms",
            "How to stay motivated while learning online",
            "Advantages of e-learning"
        ],
        "responses": [
            "Online learning platforms like Coursera, Udemy, and edX offer a wide range of courses on various subjects.",
            "Staying motivated while learning online can be achieved by setting clear learning goals, tracking progress, and participating in online communities."
        ]
    },
    {
        "tag": "art_and_crafts",
        "patterns": [
            "Tell me about art and craft ideas",
            "How to start a DIY project",
            "Benefits of creativity"
        ],
        "responses": [
            "Art and craft ideas can include painting, drawing, knitting, or making handmade cards.",
            "Creativity enhances problem-solving skills, reduces stress, and boosts self-esteem."
        ]
    },
    {
        "tag": "fashion_trends",
        "patterns": [
            "Tell me about fashion trends",
            "How to build a versatile wardrobe",
            "Sustainable fashion"
        ],
        "responses": [
            "Fashion trends change over time, but having classic pieces like a white shirt and black pants can create a versatile wardrobe.",
            "Sustainable fashion involves choosing eco-friendly materials, supporting ethical brands, and recycling clothing."
        ]
    },
    {
        "tag": "budgeting_tips",
        "patterns": [
            "Tell me about frugal living",
            "How to save money on groceries",
            "Managing finances as a student"
        ],
        "responses": [
            "Frugal living involves being mindful of expenses, budgeting, and finding ways to save money without compromising on needs.",
            "To save money on groceries, make a shopping list, compare prices, and look for discounts or coupons."
        ]
    },
    {
        "tag": "personal_finances",
        "patterns": [
            "Tell me about financial planning",
            "How to start investing",
            "Retirement planning tips"
        ],
        "responses": [
            "Financial planning involves setting financial goals, creating a budget, and building an emergency fund.",
            "Before investing, research different investment options, consider risk tolerance, and diversify your portfolio."
        ]
    },
    {
        "tag": "hobbies",
        "patterns": [
            "Tell me about interesting hobbies",
            "How to find a new hobby",
            "Benefits of pursuing hobbies"
        ],
        "responses": [
            "Interesting hobbies can include photography, gardening, playing a musical instrument, or learning a new language.",
            "Hobbies provide a creative outlet, reduce stress, and contribute to personal growth."
        ]
    },
    {
        "tag": "online_security",
        "patterns": [
            "Tell me about online security tips",
            "How to create strong passwords",
            "Protecting children online"
        ],
        "responses": [
            "Online security tips include using strong passwords, enabling two-factor authentication, and avoiding public Wi-Fi for sensitive transactions.",
            "To protect children online, use parental control software, educate them about online safety, and monitor their online activities."
        ]
    },
    {
        "tag": "home_repair",
        "patterns": [
            "Tell me about common home repairs",
            "How to fix a leaky faucet",
            "Hiring professionals for home improvement"
        ],
        "responses": [
            "Common home repairs include fixing a leaky faucet, unclogging drains, and patching small holes in walls.",
            "For complex home improvements, hiring a professional can ensure the work is done safely and effectively."
        ]
    },
    {
        "tag": "time_for_yourself",
        "patterns": [
            "How to make time for yourself",
            "Importance of self-care",
            "Relaxation techniques"
        ],
        "responses": [
            "Making time for yourself is essential for mental well-being. Set aside moments for relaxation, hobbies, or simply doing nothing.",
            "Self-care involves nurturing your physical, emotional, and mental health to maintain balance and prevent burnout."
        ]
    },
    {
        "tag": "volunteering",
        "patterns": [
            "Tell me about volunteering opportunities",
            "How to get involved in community service",
            "Benefits of volunteering"
        ],
        "responses": [
            "Volunteering opportunities can be found in local organizations, shelters, and community events.",
            "Volunteering not only helps others but also gives you a sense of purpose and fulfillment."
        ]
    },
    {
        "tag": "creative_writing",
        "patterns": [
            "Tell me about creative writing",
            "How to overcome writer's block",
            "Starting a blog"
        ],
        "responses": [
            "Creative writing includes storytelling, poetry, and fictional works. To overcome writer's block, take breaks, write regularly, and seek inspiration from other authors.",
            "Starting a blog allows you to share your thoughts, knowledge, and passions with a wider audience."
        ]
    },
    {
        "tag": "relationship_advice",
        "patterns": [
            "Tell me about relationship advice",
            "How to communicate effectively in a relationship",
            "Building trust"
        ],
        "responses": [
            "Relationship advice involves open communication, active listening, and expressing feelings and needs.",
            "Building trust in a relationship requires honesty, reliability, and being there for each other."
        ]
    },
    {
        "tag": "home_organization",
        "patterns": [
            "Tell me about home organization tips",
            "How to declutter your space",
            "Organizing small living spaces"
        ],
        "responses": [
            "Home organization tips include decluttering regularly, using storage solutions, and assigning specific places for items.",
            "For small living spaces, use multi-functional furniture and maximize vertical storage."
        ]
    },
    {
        "tag": "study_tips",
        "patterns": [
            "Tell me about effective study techniques",
            "How to stay focused while studying",
            "Preparing for exams"
        ],
        "responses": [
            "Effective study techniques include active learning, creating study schedules, and taking breaks to avoid burnout.",
            "To stay focused while studying, find a quiet and comfortable environment, avoid distractions, and set realistic goals."
        ]
    },
    {
        "tag": "positive_affirmations",
        "patterns": [
            "Tell me about positive affirmations",
            "How to practice self-affirmation",
            "Affirmations for confidence"
        ],
        "responses": [
            "Positive affirmations are powerful statements that promote self-belief and self-empowerment.",
            "Practicing self-affirmation involves repeating positive statements daily to reinforce a positive self-image."
        ]
    },
    {
        "tag": "mindfulness",
        "patterns": [
            "Tell me about mindfulness",
            "How to practice mindfulness meditation",
            "Benefits of mindfulness"
        ],
        "responses": [
            "Mindfulness is the practice of being present and fully engaged in the present moment without judgment.",
            "Mindfulness meditation can reduce stress, improve focus, and enhance emotional well-being."
        ]
    },
    {
        "tag": "car_maintenance",
        "patterns": [
            "Tell me about car maintenance tips",
            "How to change a flat tire",
            "Regular car inspections"
        ],
        "responses": [
            "Car maintenance tips include regular oil changes, checking tire pressure, and keeping fluids topped up.",
            "Knowing how to change a flat tire is a valuable skill that can come in handy during emergencies."
        ]
    },
    {
        "tag": "positivity_in_life",
        "patterns": [
            "How to cultivate positivity in life",
            "Embracing a positive mindset",
            "Seeing challenges as opportunities"
        ],
        "responses": [
            "Cultivating positivity involves focusing on gratitude, surrounding yourself with positive influences, and reframing negative thoughts.",
            "A positive mindset can improve resilience and help you approach challenges with optimism."
        ]
    },
    {
        "tag": "public_speaking",
        "patterns": [
            "Tell me about public speaking tips",
            "How to overcome stage fright",
            "Improving presentation skills"
        ],
        "responses": [
            "Public speaking tips include practicing, knowing your audience, and using visual aids to enhance your presentation.",
            "To overcome stage fright, visualize success, practice deep breathing, and start with smaller speaking engagements."
        ]
    },
    {
        "tag": "healthy_sleep",
        "patterns": [
            "Tell me about healthy sleep habits",
            "How to improve sleep quality",
            "Importance of a bedtime routine"
        ],
        "responses": [
            "Healthy sleep habits include having a consistent sleep schedule, creating a relaxing bedtime routine, and ensuring a comfortable sleep environment.",
            "Improving sleep quality can enhance mood, memory, and overall well-being."
        ]
    },
    {
        "tag": "photography",
        "patterns": [
            "Tell me about photography tips",
            "How to take better photos",
            "Different types of photography"
        ],
        "responses": [
            "Photography tips include understanding composition, lighting, and using manual settings on your camera.",
            "Different types of photography include portrait, landscape, wildlife, and street photography."
        ]
    },
    {
        "tag": "self_defense",
        "patterns": [
            "Tell me about self-defense techniques",
            "How to stay safe in public places",
            "Building self-confidence"
        ],
        "responses": [
            "Self-defense techniques involve learning basic strikes, blocks, and using your voice to create distance from potential threats.",
            "Staying safe in public places can involve being aware of your surroundings, avoiding isolated areas, and trusting your instincts."
        ]
    },
    {
        "tag": "hiking",
        "patterns": [
            "Tell me about hiking trails",
            "How to prepare for a hiking trip",
            "Benefits of hiking"
        ],
        "responses": [
            "Hiking trails vary in difficulty and can lead to beautiful landscapes, waterfalls, or scenic viewpoints.",
            "Before a hiking trip, research the trail, pack essential items like water, snacks, and a first aid kit, and let someone know your hiking plans."
        ]
    },
    {
        "tag": "nutrition_tips",
        "patterns": [
            "Tell me about healthy eating habits",
            "How to plan balanced meals",
            "Benefits of eating fruits and vegetables"
        ],
        "responses": [
            "Healthy eating habits involve incorporating a variety of fruits, vegetables, whole grains, and lean proteins in your diet.",
            "Balanced meals provide essential nutrients, energy, and support overall well-being."
        ]
    },
    {
        "tag": "positive_relationships",
        "patterns": [
            "How to build positive relationships",
            "Importance of empathy and compassion",
            "Resolving conflicts"
        ],
        "responses": [
            "Building positive relationships involves being supportive, showing empathy, and communicating openly and honestly.",
            "Empathy and compassion are essential for understanding others' perspectives and fostering strong connections."
        ]
    },
    {
        "tag": "home_fitness",
        "patterns": [
            "Tell me about home workout equipment",
            "Creating a home gym",
            "Staying fit at home"
        ],
        "responses": [
            "Home workout equipment can include resistance bands, dumbbells, yoga mats, or a stationary bike.",
            "Creating a home gym allows you to exercise conveniently and consistently without leaving your house."
        ]
    },
    {
        "tag": "job_interview_tips",
        "patterns": [
            "Tell me about job interview preparation",
            "How to answer common interview questions",
            "Dressing for job interviews"
        ],
        "responses": [
            "Job interview preparation involves researching the company, practicing answers to common questions, and preparing questions to ask the interviewer.",
            "Dressing professionally for job interviews shows respect and helps create a positive first impression."
        ]
    },
    {
        "tag": "baking",
        "patterns": [
            "Tell me about baking tips",
            "How to make homemade bread",
            "Delicious cake recipes"
        ],
        "responses": [
            "Baking tips include measuring ingredients accurately, following recipes, and understanding baking terms like creaming and folding.",
            "Homemade bread can be made with simple ingredients, time, and patience. For cakes, try classic flavors like chocolate or vanilla, or experiment with fruity combinations."
        ]
    },
    {
        "tag": "leadership_skills",
        "patterns": [
            "Tell me about developing leadership skills",
            "How to lead a team effectively",
            "Leadership qualities"
        ],
        "responses": [
            "Developing leadership skills involves continuous learning, setting a positive example, and empowering others.",
            "To lead a team effectively, communicate clearly, delegate tasks, and foster a collaborative and inclusive environment."
        ]
    },
    {
        "tag": "job_search",
        "patterns": [
            "Tell me about job search strategies",
            "How to write a compelling resume",
            "Using LinkedIn for job hunting"
        ],
        "responses": [
            "Job search strategies include networking, attending job fairs, and using online job boards.",
            "A compelling resume highlights relevant skills, experiences, and achievements. LinkedIn can be a valuable tool for connecting with potential employers."
        ]
    },
    {
        "tag": "goal_setting",
        "patterns": [
            "Tell me about setting and achieving goals",
            "How to break down big goals into smaller tasks",
            "Staying motivated during goal pursuit"
        ],
        "responses": [
            "Setting specific, measurable, achievable, relevant, and time-bound (SMART) goals increases the likelihood of success.",
            "Breaking down big goals into smaller tasks makes them more manageable and provides a sense of progress."
        ]
    },
    {
        "tag": "food_dietary_restrictions",
        "patterns": [
            "Tell me about dietary restrictions",
            "How to accommodate food allergies at gatherings",
            "Vegan and vegetarian options"
        ],
        "responses": [
            "Dietary restrictions can be due to allergies, intolerances, religious beliefs, or ethical choices.",
            "Accommodating food allergies at gatherings involves clear communication, label reading, and preparing separate dishes for allergic individuals."
        ]
    },
    {
        "tag": "technology_addiction",
        "patterns": [
            "Tell me about managing technology addiction",
            "Setting boundaries with digital devices",
            "Digital detox tips"
        ],
        "responses": [
            "Managing technology addiction involves setting designated tech-free times, using screen time trackers, and finding alternative activities.",
            "Setting boundaries with digital devices can improve sleep, productivity, and overall well-being."
        ]
    },
    {
        "tag": "pet_training",
        "patterns": [
            "Tell me about pet training tips",
            "How to teach basic commands to dogs",
            "Crate training for puppies"
        ],
        "responses": [
            "Pet training tips include using positive reinforcement, consistency, and patience to teach new behaviors.",
            "Crate training for puppies helps with house training, provides a safe space, and prevents destructive behavior."
        ]
    },
    {
        "tag": "singing",
        "patterns": [
            "Tell me about singing techniques",
            "How to warm up before singing",
            "Joining a singing group"
        ],
        "responses": [
            "Singing techniques involve proper breathing, vocal exercises, and improving pitch and tone.",
            "Warming up before singing helps prevent strain and prepares your voice for optimal performance."
        ]
    },
    {
        "tag": "plastic_waste_reduction",
        "patterns": [
            "Tell me about reducing plastic waste",
            "Using reusable alternatives",
            "Benefits of recycling"
        ],
        "responses": [
            "Reducing plastic waste can be achieved by using reusable water bottles, shopping bags, and containers.",
            "Recycling plastic reduces the amount of plastic pollution and conserves natural resources."
        ]
    },
    {
        "tag": "online_business",
        "patterns": [
            "Tell me about starting an online business",
            "How to build an e-commerce website",
            "Social media marketing"
        ],
        "responses": [
            "Starting an online business involves identifying a niche, creating a business plan, and choosing an e-commerce platform.",
            "Building an e-commerce website requires user-friendly design, secure payment options, and product descriptions."
        ]
    },
    {
        "tag": "yoga",
        "patterns": [
            "Tell me about different yoga styles",
            "How to start practicing yoga",
            "Benefits of yoga for mental health"
        ],
        "responses": [
            "Yoga styles range from Hatha, Vinyasa, to Ashtanga and offer various levels of intensity and focus.",
            "Starting yoga involves finding a suitable class or online tutorials and listening to your body during practice."
        ]
    },
    {
        "tag": "staying_connected",
        "patterns": [
            "Tell me about staying connected with loved ones",
            "Importance of maintaining relationships",
            "Long-distance friendships"
        ],
        "responses": [
            "Staying connected with loved ones involves regular calls, video chats, and planning visits when possible.",
            "Maintaining relationships fosters emotional support, happiness, and a sense of belonging."
        ]
    },
    {
        "tag": "eco-friendly_transportation",
        "patterns": [
            "Tell me about eco-friendly transportation options",
            "Advantages of cycling or walking",
            "Using public transportation"
        ],
        "responses": [
            "Eco-friendly transportation options include cycling, walking, using public transportation, or carpooling.",
            "Cycling or walking benefits the environment and promotes physical activity."
        ]
    },
    {
        "tag": "positive_self-talk",
        "patterns": [
            "Tell me about positive self-talk",
            "How to practice self-compassion",
            "Building self-confidence"
        ],
        "responses": [
            "Positive self-talk involves replacing negative thoughts with supportive and affirming ones.",
            "Practicing self-compassion involves treating yourself with kindness and understanding during challenging times."
        ]
    },
    {
        "tag": "composting",
        "patterns": [
            "Tell me about composting",
            "How to start a compost pile",
            "Benefits of composting"
        ],
        "responses": [
            "Composting is the process of recycling organic waste into nutrient-rich soil conditioner.",
            "Composting reduces food waste, enriches soil, and supports sustainable gardening."
        ]
    },
    {
        "tag": "digital_organization",
        "patterns": [
            "Tell me about organizing digital files",
            "Creating folders and file naming conventions",
            "Backup and data protection"
        ],
        "responses": [
            "Organizing digital files involves creating folders, using descriptive file names, and decluttering regularly.",
            "Backing up data is crucial to prevent data loss in case of hardware failure or accidents."
        ]
    },
    {
        "tag": "mindful_eating",
        "patterns": [
            "Tell me about mindful eating",
            "How to eat mindfully",
            "Benefits of mindful eating"
        ],
        "responses": [
            "Mindful eating involves savoring each bite, eating slowly, and paying attention to hunger and fullness cues.",
            "Practicing mindful eating can help improve digestion, prevent overeating, and enhance enjoyment of meals."
        ]
    },
    {
        "tag": "emotional_intelligence",
        "patterns": [
            "Tell me about emotional intelligence",
            "How to develop emotional intelligence",
            "Benefits of emotional intelligence in relationships"
        ],
        "responses": [
            "Emotional intelligence involves understanding and managing emotions in oneself and others.",
            "Developing emotional intelligence can lead to better communication, empathy, and healthier relationships."
        ]
    },
    {
        "tag": "home_security",
        "patterns": [
            "Tell me about home security measures",
            "Installing a security system",
            "Creating a safe home environment"
        ],
        "responses": [
            "Home security measures include installing locks, security cameras, and motion sensor lights.",
            "A security system can provide peace of mind and deter potential intruders."
        ]
    },
    {
        "tag": "memory_improvement",
        "patterns": [
            "Tell me about memory improvement techniques",
            "How to enhance memory retention",
            "Brain exercises"
        ],
        "responses": [
            "Memory improvement techniques include visualization, association, and regular mental exercises.",
            "Practicing brain exercises, like puzzles or memory games, can sharpen cognitive abilities."
        ]
    },
    {
        "tag": "social_media_detox",
        "patterns": [
            "Tell me about social media detox",
            "How to reduce social media usage",
            "Benefits of unplugging from social media"
        ],
        "responses": [
            "A social media detox involves taking a break from social media platforms for mental and emotional well-being.",
            "Reducing social media usage can improve focus, productivity, and sleep."
        ]
    },
    {
        "tag": "pet_adoption",
        "patterns": [
            "Tell me about pet adoption",
            "How to choose the right pet",
            "Responsibilities of pet ownership"
        ],
        "responses": [
            "Pet adoption provides a loving home to animals in need and brings joy and companionship to families.",
            "Choosing the right pet involves considering lifestyle, space, and the time you can dedicate to care."
        ]
    },
    {
        "tag": "interpersonal_skills",
        "patterns": [
            "Tell me about interpersonal skills",
            "How to improve communication skills",
            "Respecting diverse perspectives"
        ],
        "responses": [
            "Interpersonal skills involve active listening, empathy, and adaptability in social interactions.",
            "Improving communication skills can enhance relationships and minimize misunderstandings."
        ]
    },
    {
        "tag": "reducing_energy_consumption",
        "patterns": [
            "Tell me about reducing energy consumption",
            "Energy-saving tips at home",
            "Benefits of energy conservation"
        ],
        "responses": [
            "Reducing energy consumption involves using energy-efficient appliances, turning off lights when not in use, and properly insulating your home.",
            "Energy conservation helps lower utility bills and reduces the environmental impact."
        ]
    },
    {
        "tag": "language_learning",
        "patterns": [
            "Tell me about language learning tips",
            "How to practice a new language daily",
            "Benefits of being multilingual"
        ],
        "responses": [
            "Language learning tips include immersing yourself in the language, practicing regularly, and using language-learning apps or resources.",
            "Being multilingual can expand career opportunities, improve cognitive function, and enhance cultural understanding."
        ]
    },
    {
        "tag": "self_defense_for_women",
        "patterns": [
            "Tell me about self-defense for women",
            "Empowering women with self-defense skills",
            "Safety tips for women"
        ],
        "responses": [
            "Self-defense for women involves learning techniques to protect oneself and create a sense of empowerment.",
            "Safety tips for women include being aware of surroundings, trusting instincts, and avoiding risky situations."
        ]
    },
    {
        "tag": "healthy_relationship_with_food",
        "patterns": [
            "Tell me about having a healthy relationship with food",
            "Overcoming disordered eating patterns",
            "Balanced approach to nutrition"
        ],
        "responses": [
            "Having a healthy relationship with food involves listening to hunger and fullness cues, practicing intuitive eating, and avoiding restrictive diets.",
            "Overcoming disordered eating patterns may require professional support and a compassionate approach towards oneself."
        ]
    },
    {
        "tag": "procrastination",
        "patterns": [
            "Tell me about overcoming procrastination",
            "Tips to stay focused and productive",
            "Breaking the cycle of procrastination"
        ],
        "responses": [
            "Overcoming procrastination involves breaking tasks into smaller steps, setting deadlines, and using time management techniques.",
            "To stay focused and productive, create a conducive work environment, eliminate distractions, and reward yourself for completing tasks."
        ]
    },
    {
        "tag": "daily_meditation",
        "patterns": [
            "Tell me about daily meditation practice",
            "Creating a meditation routine",
            "Benefits of regular meditation"
        ],
        "responses": [
            "Daily meditation practice involves setting aside time each day for quiet reflection, mindfulness, or deep breathing exercises.",
            "Regular meditation can reduce stress, improve mental clarity, and enhance overall well-being."
        ]
    },
    {
        "tag": "charity_donations",
        "patterns": [
            "Tell me about charity donations",
            "Choosing a reputable charity organization",
            "Ways to support charitable causes"
        ],
        "responses": [
            "Charity donations support various causes, such as education, health, environmental conservation, and humanitarian aid.",
            "Before donating, research and verify the legitimacy and impact of the charity organization."
        ]
    },
    {
        "tag": "resilience",
        "patterns": [
            "Tell me about building resilience",
            "Coping with setbacks and challenges",
            "Developing emotional strength"
        ],
        "responses": [
            "Building resilience involves accepting change, developing problem-solving skills, and seeking support from others.",
            "Coping with setbacks requires self-compassion and a focus on learning and growth."
        ]
    },
    {
        "tag": "meal_preparation",
        "patterns": [
            "Tell me about meal preparation tips",
            "How to plan and prep meals in advance",
            "Benefits of meal prepping"
        ],
        "responses": [
            "Meal preparation tips include planning meals, batch cooking, and using reusable meal containers.",
            "Meal prepping saves time, reduces food waste, and promotes healthier eating habits."
        ]
    },
    {
        "tag": "work_life_balance",
        "patterns": [
            "Tell me about achieving work-life balance",
            "Setting boundaries between work and personal life",
            "The importance of self-care in maintaining balance"
        ],
        "responses": [
            "Achieving work-life balance involves prioritizing personal time, setting boundaries, and knowing when to take breaks.",
            "Self-care is essential for maintaining balance and preventing burnout."
        ]
    },
    {
        "tag": "mindfulness_in_nature",
        "patterns": [
            "Tell me about mindfulness in nature",
            "Practicing nature meditation",
            "Connecting with the natural world"
        ],
        "responses": [
            "Mindfulness in nature involves being present and fully experiencing the sights, sounds, and sensations of the natural environment.",
            "Nature meditation can reduce stress and foster a deeper connection with the outdoors."
        ]
    },
    {
        "tag": "journaling",
        "patterns": [
            "Tell me about journaling benefits",
            "How to start a journaling practice",
            "Different journaling styles"
        ],
        "responses": [
            "Journaling benefits include stress relief, self-reflection, and gaining insights into thoughts and emotions.",
            "Starting a journaling practice can involve writing freely, using prompts, or maintaining a gratitude journal."
        ]
    },
    {
        "tag": "reducing_single_use_plastics",
        "patterns": [
            "Tell me about reducing single-use plastics",
            "Using eco-friendly alternatives",
            "Supporting plastic-free initiatives"
        ],
        "responses": [
            "Reducing single-use plastics involves using reusable water bottles, coffee cups, and shopping bags.",
            "Supporting plastic-free initiatives and advocating for sustainable practices can drive positive change."
        ]
    },
    {
        "tag": "team_building",
        "patterns": [
            "Tell me about team-building activities",
            "How to foster a positive team dynamic",
            "Improving communication within teams"
        ],
        "responses": [
            "Team-building activities can include team retreats, problem-solving exercises, and icebreaker games.",
            "Fostering a positive team dynamic requires open communication, trust, and recognizing individual strengths."
        ]
    },
    {
        "tag": "financial_investing",
        "patterns": [
            "Tell me about financial investing",
            "Understanding stocks, bonds, and mutual funds",
            "Creating a diversified investment portfolio"
        ],
        "responses": [
            "Financial investing involves putting money into assets like stocks, bonds, and mutual funds with the expectation of generating returns over time.",
            "Diversifying your investment portfolio can help manage risk and optimize returns."
        ]
    },
    {
        "tag": "reusable_transportation",
        "patterns": [
            "Tell me about using reusable transportation",
            "Benefits of cycling or walking for commuting",
            "Car-sharing and ride-sharing services"
        ],
        "responses": [
            "Using reusable transportation, like cycling or walking, reduces carbon emissions and promotes physical fitness.",
            "Car-sharing and ride-sharing services provide convenient alternatives to private car ownership."
        ]
    },
    {
        "tag": "generosity_and_giving",
        "patterns": [
            "Tell me about the importance of generosity and giving",
            "Ways to give back to the community",
            "Volunteer opportunities"
        ],
        "responses": [
            "Generosity and giving can create a positive impact on individuals and communities in need.",
            "Ways to give back include volunteering time, donating to charities, and supporting local causes."
        ]
    },
    {
        "tag": "energy_boosting_foods",
        "patterns": [
            "Tell me about energy-boosting foods",
            "Nutritious snacks for an energy pick-me-up",
            "Importance of staying hydrated"
        ],
        "responses": [
            "Energy-boosting foods include fruits, nuts, whole grains, and foods rich in protein and antioxidants.",
            "Snacks like trail mix, yogurt, or smoothies can provide a quick energy pick-me-up during busy days."
        ]
    },
    {
        "tag": "exploring_culture_and_traditions",
        "patterns": [
            "Tell me about exploring different cultures and traditions",
            "How to embrace diversity",
            "Benefits of cultural exchange"
        ],
        "responses": [
            "Exploring different cultures and traditions can broaden perspectives, foster empathy, and promote tolerance and understanding.",
            "Embracing diversity enriches communities and allows for a greater appreciation of the world."
        ]
    },
    {
        "tag": "effective_communication_in_relationships",
        "patterns": [
            "Tell me about effective communication in relationships",
            "Active listening and empathy",
            "Resolving conflicts peacefully"
        ],
        "responses": [
            "Effective communication in relationships involves active listening, expressing emotions constructively, and validating each other's feelings.",
            "Practicing empathy and resolving conflicts peacefully strengthen emotional bonds."
        ]
    },
    {
        "tag": "cooking_for_special_occasions",
        "patterns": [
            "Tell me about cooking for special occasions",
            "Creating a memorable dinner party menu",
            "Dessert recipes for celebrations"
        ],
        "responses": [
            "Cooking for special occasions allows you to showcase your culinary skills and create unforgettable dining experiences.",
            "When planning a dinner party menu, consider your guests' preferences and dietary restrictions."
        ]
    },
    {
        "tag": "developing_creativity_in_children",
        "patterns": [
            "Tell me about developing creativity in children",
            "Art and craft activities for kids",
            "The importance of unstructured play"
        ],
        "responses": [
            "Developing creativity in children involves encouraging imaginative play, providing art supplies, and fostering curiosity and exploration.",
            "Art and craft activities can enhance fine motor skills, self-expression, and cognitive development."
        ]
    },
    {
        "tag": "emotional_resilience_in_children",
        "patterns": [
            "Tell me about fostering emotional resilience in children",
            "Teaching coping skills for challenges",
            "Building a supportive and nurturing environment"
        ],
        "responses": [
            "Fostering emotional resilience in children involves validating their feelings, teaching problem-solving, and helping them develop a growth mindset.",
            "A supportive and nurturing environment at home and school contributes to emotional well-being."
        ]
    },
    {
        "tag": "self_reflection",
        "patterns": [
            "Tell me about the importance of self-reflection",
            "Journaling for self-discovery",
            "Embracing personal growth"
        ],
        "responses": [
            "Self-reflection is essential for gaining self-awareness, identifying strengths and areas for growth, and making positive changes in life.",
            "Journaling can be a powerful tool for self-discovery, emotional processing, and goal-setting."
        ]
    },
    {
        "tag": "mindful_technology_usage",
        "patterns": [
            "Tell me about mindful technology usage",
            "Setting digital boundaries for a healthy lifestyle",
            "Balancing screen time with real-world experiences"
        ],
        "responses": [
            "Mindful technology usage involves being intentional with screen time, setting boundaries, and using technology in ways that support well-being.",
            "Balancing screen time with real-world experiences fosters social connections and reduces digital overwhelm."
        ]
    },
    {
        "tag": "youth_mental_health",
        "patterns": [
            "Tell me about youth mental health",
            "Supporting teenagers' emotional well-being",
            "Recognizing signs of distress in young people"
        ],
        "responses": [
            "Youth mental health is crucial for overall well-being and requires understanding, support, and open communication.",
            "Supporting teenagers' emotional well-being involves being approachable, validating their feelings, and encouraging help-seeking behavior when needed."
        ]
    },
    {
        "tag": "mindful_consumption",
        "patterns": [
            "Tell me about mindful consumption",
            "Practicing sustainable shopping habits",
            "Responsible use of resources"
        ],
        "responses": [
            "Mindful consumption involves being conscious of your purchasing choices, buying only what you need, and considering the environmental impact of products.",
            "Practicing sustainable shopping habits, like buying second-hand items and supporting eco-friendly brands, can contribute to reducing waste and promoting ethical production."
        ]
    },
    {
        "tag": "effective_study_groups",
        "patterns": [
            "Tell me about effective study groups",
            "Benefits of group studying",
            "Setting study goals as a team"
        ],
        "responses": [
            "Effective study groups can enhance learning through peer discussions, explaining concepts to others, and sharing study resources.",
            "Setting study goals as a team and holding each other accountable can lead to improved academic performance."
        ]
    },
    {
        "tag": "social_anxiety",
        "patterns": [
            "Tell me about coping with social anxiety",
            "Practicing relaxation techniques for social situations",
            "Seeking professional support for anxiety disorders"
        ],
        "responses": [
            "Coping with social anxiety involves challenging negative thoughts, gradual exposure to social situations, and learning relaxation techniques to manage anxiety symptoms.",
            "For severe social anxiety or anxiety disorders, seeking professional support, such as therapy or counseling, can be beneficial."
        ]
    },
    {
        "tag": "responsible_pet_ownership",
        "patterns": [
            "Tell me about responsible pet ownership",
            "Importance of regular vet check-ups",
            "Creating a safe and enriching environment for pets"
        ],
        "responses": [
            "Responsible pet ownership involves providing proper nutrition, regular exercise, and veterinary care for pets.",
            "Regular vet check-ups help monitor a pet's health, detect potential issues early, and ensure they are up-to-date on vaccinations."
        ]
    },
    {
        "tag": "gardening",
        "patterns": [
            "Tell me about gardening as a hobby",
            "Growing herbs and vegetables at home",
            "The therapeutic benefits of gardening"
        ],
        "responses": [
            "Gardening can be a fulfilling hobby that allows you to connect with nature, grow your own food, and create beautiful landscapes.",
            "Growing herbs and vegetables at home provides fresh and nutritious produce for your meals."
        ]
    },
    {
        "tag": "waste_reduction_at_work",
        "patterns": [
            "Tell me about waste reduction at work",
            "Encouraging recycling and proper waste disposal in the workplace",
            "Implementing sustainable practices in office settings"
        ],
        "responses": [
            "Waste reduction at work involves promoting recycling, minimizing paper usage, and encouraging employees to adopt sustainable practices.",
            "Implementing sustainable practices, like energy-efficient lighting and reducing single-use plastics, can contribute to a greener workplace."
        ]
    },
    {
        "tag": "effective_decision_making",
        "patterns": [
            "Tell me about effective decision-making",
            "Using pros and cons analysis for choices",
            "Trusting intuition in decision-making"
        ],
        "responses": [
            "Effective decision-making involves gathering information, weighing pros and cons, and considering the potential outcomes.",
            "Trusting intuition in decision-making can be valuable, especially when facing complex choices."
        ]
    },
    {
        "tag": "alien_invasion",
        "patterns": [
            "What would you do during an alien invasion?",
            "How to prepare for an alien attack?",
            "Best escape plan during an alien invasion"
        ],
        "responses": [
            "In case of an alien invasion, it's best to find a safe shelter and avoid attracting attention.",
            "Preparing for an alien attack may involve stocking up on supplies and creating a communication plan with loved ones."
        ]
    },
    {
        "tag": "space_travel",
        "patterns": [
            "Tell me about traveling to space",
            "How to become an astronaut",
            "Best space destinations for tourists"
        ],
        "responses": [
            "Space travel is a fascinating experience but requires rigorous training and physical fitness to become an astronaut.",
            "Space tourism is an emerging industry, and some private companies offer opportunities for civilians to experience space travel."
        ]
    },
    {
        "tag": "ninja_skills",
        "patterns": [
            "How to learn ninja skills",
            "Ninja training techniques",
            "Secrets of stealth and invisibility"
        ],
        "responses": [
            "Ninja skills are steeped in secrecy and traditionally passed down through generations.",
            "Learning ninja skills involves mastering martial arts, stealth, and discipline."
        ]
    },
    {
        "tag": "time_travel",
        "patterns": [
            "Tell me about time travel theories",
            "How to build a time machine",
            "Visiting historical events through time travel"
        ],
        "responses": [
            "Time travel remains a theoretical concept, and various scientific theories explore its possibilities.",
            "Building a time machine is currently beyond our technological capabilities, but it's a fascinating idea."
        ]
    },
    {
        "tag": "unicorn_taming",
        "patterns": [
            "How to tame a unicorn",
            "Best treats for unicorns",
            "Where to find wild unicorns"
        ],
        "responses": [
            "Taming a unicorn is a mythical endeavor, as unicorns are legendary creatures.",
            "Unicorns are said to be elusive, and their existence is a matter of folklore."
        ]
    },
    {
        "tag": "wizard_training",
        "patterns": [
            "How to become a wizard",
            "Wizardry schools and academies",
            "Essential spells for beginners"
        ],
        "responses": [
            "Becoming a wizard involves studying magical arts, honing spellcasting skills, and understanding the mystical world.",
            "Wizardry schools, like Hogwarts in Harry Potter, are iconic in fiction but do not exist in the real world."
        ]
    },
    {
        "tag": "chocolate_obsession",
        "patterns": [
            "Tell me about a chocolate obsession",
            "Indulging in chocolate guilt-free",
            "Chocolate-themed parties and events"
        ],
        "responses": [
            "A chocolate obsession is common, given the deliciousness of this sweet treat!",
            "Enjoying chocolate in moderation can be a delightful and guilt-free experience."
        ]
    },
    {
        "tag": "telepathy_skills",
        "patterns": [
            "How to develop telepathy skills",
            "Communicating through telepathy",
            "Experiencing mind-reading abilities"
        ],
        "responses": [
            "Telepathy is a paranormal phenomenon, and there is no scientific evidence to support its existence.",
            "The concept of telepathy has captivated human imagination for centuries."
        ]
    },
    {
        "tag": "butterfly_language",
        "patterns": [
            "Understanding the language of butterflies",
            "Talking to butterflies for relaxation",
            "Interpreting butterfly movements"
        ],
        "responses": [
            "Butterflies do not have a formal language, but observing their behavior can be a peaceful and meditative experience.",
            "Butterflies' vibrant colors and graceful flights are nature's marvels."
        ]
    },
    {
        "tag": "astral_projection",
        "patterns": [
            "Tell me about astral projection",
            "How to have an out-of-body experience",
            "Exploring the astral plane"
        ],
        "responses": [
            "Astral projection is an esoteric concept where one's consciousness is believed to travel outside the physical body.",
            "Astral projection is a mystical idea explored in spiritual practices and metaphysical beliefs."
        ]
    },
    {
        "tag": "dragon_riding",
        "patterns": [
            "Tell me about riding a dragon",
            "How to befriend a dragon",
            "Visiting dragon sanctuaries"
        ],
        "responses": [
            "Riding a dragon is a magical notion found in mythology and fantasy tales.",
            "Dragons are mythical creatures often depicted as powerful and awe-inspiring."
        ]
    },
    {
        "tag": "cloud_castles",
        "patterns": [
            "Building castles in the clouds",
            "Cloud castle architecture",
            "Living in cloud cities"
        ],
        "responses": [
            "Building castles in the clouds is an imaginative expression for daydreaming or envisioning the impossible.",
            "Cloud cities and castles remain confined to dreams and fictional stories."
        ]
    },
    {
        "tag": "plant_whispering",
        "patterns": [
            "How to communicate with plants",
            "Listening to plants' whispers",
            "Plant emotions and feelings"
        ],
        "responses": [
            "Plant whispering is a metaphorical concept that emphasizes the importance of caring for and understanding plants.",
            "Plants respond positively to proper care and nurturing, although they do not communicate through language."
        ]
    },
    {
        "tag": "pirate_treasures",
        "patterns": [
            "Tell me about hidden pirate treasures",
            "Where to find buried pirate loot",
            "Pirate maps and clues"
        ],
        "responses": [
            "Hidden pirate treasures are legendary tales that have captivated people's imaginations for centuries.",
            "Pirate folklore often includes stories of buried treasure chests and treasure maps."
        ]
    },
    {
        "tag": "underwater_exploration",
        "patterns": [
            "Tell me about exploring the depths of the ocean",
            "Discovering underwater civilizations",
            "Encounters with mythical sea creatures"
        ],
        "responses": [
            "Underwater exploration reveals the fascinating marine life and ecosystems that exist beneath the ocean's surface.",
            "Mythical sea creatures, like mermaids and sea monsters, are a part of folklore and legends."
        ]
    },
    {
        "tag": "alien_languages",
        "patterns": [
            "Learning alien languages",
            "Translating extraterrestrial communications",
            "Universal communication with aliens"
        ],
        "responses": [
            "Alien languages are a fictional construct used in science fiction stories and movies.",
            "If aliens exist, communicating with them would be a complex and challenging task."
        ]
    },
    {
        "tag": "enchanted_forests",
        "patterns": [
            "Tell me about enchanted forests",
            "Magical creatures in the woods",
            "Seeking magical wisdom from forest spirits"
        ],
        "responses": [
            "Enchanted forests are often depicted as mysterious and magical places in folklore and fairy tales.",
            "Folklore and mythology abound with stories of mystical creatures and spirits residing in forests."
        ]
    },
    {
        "tag": "invisibility_cloak",
        "patterns": [
            "How to make an invisibility cloak",
            "Mastering the art of invisibility",
            "Adventures with an invisible cloak"
        ],
        "responses": [
            "Invisibility cloaks are fantastical objects found in fantasy literature, not in the real world.",
            "The idea of invisibility has intrigued humanity throughout history."
        ]
    },
    {
        "tag": "talking_animals",
        "patterns": [
            "Tell me about talking animals",
            "Conversations with intelligent animals",
            "The language of animals"
        ],
        "responses": [
            "Talking animals are a common theme in children's stories and animated movies.",
            "In reality, animals communicate through body language, vocalizations, and other non-verbal cues."
        ]
    },
    {
        "tag": "robot_companions",
        "patterns": [
            "Having a robot companion",
            "Choosing the perfect robot friend",
            "Adventures with a robotic buddy"
        ],
        "responses": [
            "Robot companions are a concept explored in science fiction, where robots are designed to interact with humans as friends or helpers.",
            "In the real world, robots are used for practical purposes like automation and assistance in industries."
        ]
    },
    {
        "tag": "magic_school",
        "patterns": [
            "Tell me about attending a magic school",
            "Enrolling in a school of sorcery",
            "Classes in magical arts"
        ],
        "responses": [
            "Magic schools, like the ones depicted in fantasy books and movies, are fictional settings for learning spells and wizardry.",
            "In real life, education focuses on science, technology, arts, and other practical subjects."
        ]
    },
    {
        "tag": "enchanted_potions",
        "patterns": [
            "Brewing enchanted potions",
            "Ingredients for magical elixirs",
            "Potions for love and luck"
        ],
        "responses": [
            "Enchanted potions are a common element in magical tales, capable of granting wishes or causing transformations.",
            "In reality, potions and elixirs are symbolic representations used in rituals and cultural practices."
        ]
    },
    {
        "tag": "treasure_hunting",
        "patterns": [
            "Tell me about treasure hunting",
            "Following ancient maps to hidden riches",
            "The excitement of discovering buried treasure"
        ],
        "responses": [
            "Treasure hunting is a thrilling pursuit depicted in adventure stories and historical tales.",
            "Real-world treasure hunting involves searching for historical artifacts and archaeological discoveries."
        ]
    },
    {
        "tag": "paranormal_investigation",
        "patterns": [
            "How to investigate paranormal activity",
            "Encounters with ghosts and spirits",
            "Haunted locations to explore"
        ],
        "responses": [
            "Paranormal investigation involves exploring phenomena like ghosts, poltergeists, and supernatural events.",
            "People who engage in paranormal investigation are often intrigued by the unknown and unexplained."
        ]
    },
    {
        "tag": "superhero_training",
        "patterns": [
            "Tell me about superhero training",
            "Becoming a real-life superhero",
            "Discovering hidden superpowers"
        ],
        "responses": [
            "Superhero training is a popular theme in comic books and superhero movies, showcasing characters' journeys to become powerful defenders of justice.",
            "In reality, ordinary people can become heroes through acts of kindness, bravery, and compassion."
        ]
    },
    {
        "tag": "time_bending",
        "patterns": [
            "Tell me about bending time",
            "Manipulating time for travel and exploration",
            "The science behind time manipulation"
        ],
        "responses": [
            "Bending time is a concept often explored in science fiction and speculative fiction.",
            "Time is a fundamental dimension in physics, and its manipulation remains theoretical."
        ]
    },
    {
        "tag": "extraterrestrial_cuisine",
        "patterns": [
            "Trying extraterrestrial cuisine",
            "Unique flavors from outer space",
            "Preparing alien-inspired dishes"
        ],
        "responses": [
            "Extraterrestrial cuisine is an imaginative concept that explores what food might be like on other planets.",
            "While we have not encountered aliens, culinary creativity allows us to imagine otherworldly flavors."
        ]
    },
    {
        "tag": "dream_world",
        "patterns": [
            "Tell me about the dream world",
            "Lucid dreaming and controlling dreams",
            "Creating dreamscapes"
        ],
        "responses": [
            "The dream world is a realm of imagination and subconscious experiences that occur during sleep.",
            "Lucid dreaming is a phenomenon where individuals are aware that they are dreaming and can sometimes influence the dream's content."
        ]
    },
    {
        "tag": "book_of_spells",
        "patterns": [
            "Possessing a book of magical spells",
            "Unleashing the power of ancient incantations",
            "Collecting rare and powerful spells"
        ],
        "responses": [
            "The idea of a book of spells is a popular trope in fantasy literature, where mystical tomes contain powerful incantations and enchantments.",
            "In reality, books are valuable sources of knowledge and information."
        ]
    },
    {
        "tag": "space_dance_party",
        "patterns": [
            "Hosting a dance party in space",
            "Zero-gravity dance moves",
            "Inviting aliens to a cosmic disco"
        ],
        "responses": [
            "A space dance party is an imaginative idea where people groove to music amidst the stars.",
            "While there are no dance parties in space, astronauts do exercises to stay fit in microgravity."
        ]
    },
    {
        "tag": "rainbow_treasure",
        "patterns": [
            "Hunting for the end of a rainbow",
            "Discovering a pot of gold",
            "The secrets of rainbow lore"
        ],
        "responses": [
            "The end of a rainbow is an optical illusion, and it's impossible to physically reach it.",
            "Rainbows are natural phenomena caused by the refraction and dispersion of light."
        ]
    },
    {
        "tag": "jedi_training",
        "patterns": [
            "How to become a Jedi",
            "Mastering lightsaber combat",
            "Balancing the Force within"
        ],
        "responses": [
            "Jedi are iconic characters from the Star Wars universe, skilled in the ways of the Force and lightsaber combat.",
            "In real life, individuals can practice martial arts and mindfulness to develop discipline and inner strength."
        ]
    },
    {
        "tag": "timeless_treasures",
        "patterns": [
            "Collecting timeless treasures",
            "Exploring ancient artifacts",
            "Preserving relics from the past"
        ],
        "responses": [
            "Timeless treasures are valuable artifacts and objects that hold historical, cultural, or sentimental significance.",
            "Museums and historical sites preserve relics from the past for future generations."
        ]
    },
    {
        "tag": "mermaid_encounters",
        "patterns": [
            "Tell me about encountering mermaids",
            "Swimming with merfolk in the ocean",
            "Folktales of mermaid kingdoms"
        ],
        "responses": [
            "Mermaids are mythical aquatic creatures often depicted as part-human and part-fish.",
            "Mermaid legends have been part of maritime folklore across various cultures."
        ]
    },
    {
        "tag": "mind_reading_glasses",
        "patterns": [
            "Wearing mind-reading glasses",
            "Understanding thoughts through eyewear",
            "The science of reading minds"
        ],
        "responses": [
            "Mind-reading glasses are a fantastical idea, where glasses allow wearers to perceive others' thoughts.",
            "Mind-reading is not scientifically proven, and respecting privacy is essential in human interactions."
        ]
    },
    {
        "tag": "time_warps",
        "patterns": [
            "Tell me about time warps and wormholes",
            "Time travel through cosmic anomalies",
            "Navigating through time vortexes"
        ],
        "responses": [
            "Time warps and wormholes are theoretical concepts in physics, which suggest shortcuts in space-time.",
            "The idea of time travel through wormholes has captured the imagination of scientists and science fiction enthusiasts."
        ]
    },
    {
        "tag": "fortune_cookie_messages",
        "patterns": [
            "Interpreting fortune cookie messages",
            "Fortune cookie predictions coming true",
            "Writing personalized fortune cookies"
        ],
        "responses": [
            "Fortune cookie messages are fun and often vague predictions or words of wisdom.",
            "Fortune cookies are a popular tradition in Chinese restaurants and are enjoyed as a lighthearted treat."
        ]
    },
    {
        "tag": "enchanted_music",
        "patterns": [
            "Listening to enchanted music",
            "Music that casts spells and charms",
            "Magical melodies and songs"
        ],
        "responses": [
            "Enchanted music is an imaginative concept where melodies possess mystical qualities.",
            "Music has the power to evoke emotions, memories, and a sense of wonder."
        ]
    },
    {
        "tag": "comedy_tonic",
        "patterns": [
            "Using comedy as a tonic for happiness",
            "Laughter as a remedy for stress",
            "Creating joy with humor"
        ],
        "responses": [
            "Comedy is a fantastic tonic for lifting spirits and promoting positivity.",
            "Laughter is known to have therapeutic effects and is beneficial for mental well-being."
        ]
    },
    {
        "tag": "fire-breathing_dragon",
        "patterns": [
            "Tell me about a fire-breathing dragon",
            "Dragon breath and its power",
            "Challenges of taming a dragon"
        ],
        "responses": [
            "Fire-breathing dragons are legendary creatures in myths and folklore, known for their ability to exhale fire.",
            "Taming a dragon is a perilous quest depicted in many heroic tales."
        ]
    },
    {
        "tag": "mind_control",
        "patterns": [
            "Exploring the concept of mind control",
            "Mind control in science fiction",
            "Psychic abilities and mental manipulation"
        ],
        "responses": [
            "Mind control is a sci-fi trope where individuals possess the power to control others' thoughts and actions.",
            "In reality, mind control remains a subject of science fiction and not a real-world capability."
        ]
    },
    {
        "tag": "flying_on_broomsticks",
        "patterns": [
            "Tell me about flying on broomsticks",
            "Mastering the art of broomstick riding",
            "The thrill of broomstick races"
        ],
        "responses": [
            "Flying on broomsticks is a magical ability attributed to witches and wizards in folklore and fantasy literature.",
            "Broomstick races and aerial sports are popular events in fictional wizarding worlds."
        ]
    },
    {
        "tag": "enchanted_books",
        "patterns": [
            "Discovering enchanted books",
            "Books that come to life",
            "Reading spells from magical tomes"
        ],
        "responses": [
            "Enchanted books are a common theme in fantasy, where books possess magical properties or contain hidden knowledge.",
            "Books are wonderful sources of knowledge, allowing readers to explore new worlds and ideas."
        ]
    },
    {
        "tag": "starship_adventures",
        "patterns": [
            "Tell me about starship adventures",
            "Captain of a starship crew",
            "Exploring distant galaxies"
        ],
        "responses": [
            "Starship adventures are thrilling journeys through space, encountering alien civilizations and cosmic wonders.",
            "While starships are common in science fiction, real-world space exploration involves robotic missions and astronaut expeditions."
        ]
    },
    {
        "tag": "enchanted_garden",
        "patterns": [
            "Visiting an enchanted garden",
            "Magical flora and fauna",
            "Speaking with enchanted plants"
        ],
        "responses": [
            "Enchanted gardens are mystical places filled with magical plants, flowers, and creatures.",
            "Gardens in reality are beautiful sanctuaries where people can connect with nature."
        ]
    },
    {
        "tag": "time_crystals",
        "patterns": [
            "Tell me about time crystals",
            "The mysteries of temporal crystals",
            "Time manipulation with crystal energy"
        ],
        "responses": [
            "Time crystals are theoretical structures with repeating patterns that exist in time rather than space.",
            "Time crystals are an exciting concept in quantum physics, but their practical applications are still theoretical."
        ]
    },
    {
        "tag": "magic_mirrors",
        "patterns": [
            "Using magic mirrors for communication",
            "Portals to other dimensions",
            "Mirror reflections revealing the truth"
        ],
        "responses": [
            "Magic mirrors are a common motif in folklore and fairy tales, serving as tools for communication and divination.",
            "Mirrors have long been associated with symbolism and metaphors in various cultures."
        ]
    },
    {
        "tag": "cloud_castle_ruler",
        "patterns": [
            "Becoming the ruler of a cloud castle",
            "Cloud castle governance and policies",
            "Pros and cons of cloud kingdom leadership"
        ],
        "responses": [
            "Ruling a cloud castle is a whimsical notion, given that cloud castles exist only in the realms of fantasy.",
            "Cloud castles are dreamlike representations of majestic palaces amidst the sky."
        ]
    },
    {
        "tag": "shape-shifting_abilities",
        "patterns": [
            "Tell me about shape-shifting abilities",
            "The art of transforming into other beings",
            "Limitations of shape-shifters"
        ],
        "responses": [
            "Shape-shifting is a supernatural ability often depicted in folklore, myths, and fairy tales.",
            "In reality, shape-shifting is not scientifically possible, but it remains an intriguing concept."
        ]
    },
    {
        "tag": "fantasy_adventures",
        "patterns": [
            "Embarking on epic fantasy adventures",
            "Quests for mythical artifacts and treasures",
            "Slaying dragons and vanquishing dark lords"
        ],
        "responses": [
            "Fantasy adventures take characters on thrilling quests where they encounter magical creatures and confront forces of evil.",
            "Reading fantasy books and playing role-playing games can transport individuals to imaginative worlds."
        ]
    },
    {
        "tag": "enchanted_mazes",
        "patterns": [
            "Navigating through enchanted mazes",
            "Mysteries and surprises in magical labyrinths",
            "Solving riddles in fantasy mazes"
        ],
        "responses": [
            "Enchanted mazes are intricate puzzles found in fairy tales and games, filled with hidden treasures and challenges.",
            "Real-world mazes and labyrinths are recreational attractions that offer fun and enjoyment."
        ]
    },
    {
        "tag": "mind_bending_riddles",
        "patterns": [
            "Tell me about mind-bending riddles",
            "Solving riddles to unlock secrets",
            "Riddles from ancient civilizations"
        ],
        "responses": [
            "Mind-bending riddles are puzzling questions that require clever thinking and problem-solving skills.",
            "Riddles have been used in folklore, literature, and cultural traditions to entertain and challenge minds."
        ]
    },
    {
        "tag": "wizard_apprenticeship",
        "patterns": [
            "Becoming a wizard's apprentice",
            "Learning magic under a mentor",
            "The journey to becoming a master wizard"
        ],
        "responses": [
            "Wizard apprenticeship is a classic theme in fantasy, where young characters train under experienced mentors to become skilled in magical arts.",
            "In real life, mentorship and apprenticeships exist in various fields, allowing individuals to learn from seasoned professionals."
        ]
    },
    {
        "tag": "fantasy_dreamscape",
        "patterns": [
            "Tell me about the fantasy dreamscape",
            "Exploring surreal dream worlds",
            "Creatures and landscapes in dreamscapes"
        ],
        "responses": [
            "The fantasy dreamscape is a realm of dreams and imagination, where anything is possible and surreal scenes unfold.",
            "Dreams are a mysterious aspect of human consciousness, often inspiring creativity and introspection."
        ]
    },
    {
        "tag": "time_traveler_guest",
        "patterns": [
            "Hosting a time traveler as a guest",
            "Time travelers' stories of historical events",
            "Gifts for time-traveling visitors"
        ],
        "responses": [
            "Hosting a time traveler would be an extraordinary experience, allowing for insights into historical periods and distant futures.",
            "Time travel is a popular theme in fiction, with time travelers encountering various historical figures and events."
        ]
    },
    {
        "tag": "enchanted_forest_lake",
        "patterns": [
            "Discovering an enchanted forest lake",
            "The magic of reflective waters",
            "Guardians of mystical lakes"
        ],
        "responses": [
            "Enchanted forest lakes are mythical places where the waters possess magical properties or conceal secrets.",
            "Lakes in reality hold ecological significance and are vital for supporting ecosystems."
        ]
    },
    {
        "tag": "mystical_creatures",
        "patterns": [
            "Tell me about encountering mystical creatures",
            "Interactions with mythical beings",
            "Learning from creatures of wisdom"
        ],
        "responses": [
            "Mystical creatures are fantastical beings with supernatural abilities and wisdom, often serving as guides or guardians.",
            "Throughout history, folklore and myths have featured a diverse array of mystical creatures."
        ]
    },
    {
        "tag": "timeless_puzzles",
        "patterns": [
            "Solving timeless puzzles",
            "Unraveling ancient enigmas",
            "The thrill of deciphering cryptic codes"
        ],
        "responses": [
            "Timeless puzzles are enduring challenges that test one's intellect, logic, and ingenuity.",
            "Puzzles have been popular forms of entertainment and mental exercises for millennia."
        ]
    },
    {
        "tag": "magic_cauldron",
        "patterns": [
            "Unleashing magic from a cauldron",
            "Potions and concoctions brewed in cauldrons",
            "The power of a magical cooking pot"
        ],
        "responses": [
            "Magic cauldrons are iconic symbols of witchcraft and wizardry, used for brewing potions and spells.",
            "In reality, cauldrons have historical significance and were used for cooking and various purposes in ancient times."
        ]
    },
    {
        "tag": "wizardly_quizzes",
        "patterns": [
            "Participating in wizardly quizzes",
            "Challenges and tests of magical knowledge",
            "Winning rewards from mystical quizzes"
        ],
        "responses": [
            "Wizardly quizzes are contests that assess one's knowledge of magical lore, spells, and mythical creatures.",
            "Quizzes and trivia games are popular ways to engage in fun and interactive learning."
        ]
    },
    {
        "tag": "enchanted_compass",
        "patterns": [
            "Using an enchanted compass",
            "Finding the way through magical lands",
            "The compass that points to desires"
        ],
        "responses": [
            "An enchanted compass is a fantastical device that guides travelers through mystical realms and hidden destinations.",
            "Real-world compasses have been crucial navigational tools for exploration and orientation."
        ]
    },
    {
        "tag": "arcane_alchemy",
        "patterns": [
            "Delving into the mysteries of arcane alchemy",
            "Transmuting base metals into gold",
            "The quest for the philosopher's stone"
        ],
        "responses": [
            "Arcane alchemy is an ancient practice that aimed to transform substances and uncover the secrets of existence.",
            "Alchemy was an early form of chemistry that laid the foundation for modern scientific discoveries."
        ]
    },
    {
        "tag": "spellbinding_artifacts",
        "patterns": [
            "Tell me about spellbinding artifacts",
            "Objects with magical properties",
            "Relics from legendary tales"
        ],
        "responses": [
            "Spellbinding artifacts are objects imbued with magical properties or associated with mythical legends.",
            "Museums and cultural institutions house historical artifacts that tell stories of human civilization."
        ]
    },
    {
        "tag": "enchanted_dance",
        "patterns": [
            "Dancing in an enchanted ball",
            "Enchanting melodies and dance moves",
            "Dancing with mythical partners"
        ],
        "responses": [
            "An enchanted dance ball is a magical event where music, dance, and celebration intertwine.",
            "Dance is an expressive art form that brings joy, emotional release, and cultural significance."
        ]
    },
    {
        "tag": "flying_carpet_adventures",
        "patterns": [
            "Tell me about flying carpet adventures",
            "The thrill of carpet riding",
            "Visiting far-off lands on a magic carpet"
        ],
        "responses": [
            "Flying carpet adventures are common in Arabian Nights tales and represent fantastical journeys through the sky.",
            "Real-world transportation includes cars, planes, and other vehicles that allow people to travel vast distances."
        ]
    },
    {
        "tag": "enchanted_music_box",
        "patterns": [
            "Discovering an enchanted music box",
            "Melodies that enchant the soul",
            "Music that transcends time"
        ],
        "responses": [
            "An enchanted music box is a mystical container that plays magical melodies capable of captivating hearts.",
            "Music boxes are charming keepsakes that have delighted people for centuries."
        ]
    },
    {
        "tag": "fairy_rings",
        "patterns": [
            "The mystery of fairy rings",
            "Folklore and myths surrounding fairy circles",
            "Encounters with fairies in enchanted circles"
        ],
        "responses": [
            "Fairy rings, also known as fairy circles, are naturally occurring circular patterns of mushrooms in the forest.",
            "Fairy folklore is rich with stories of ethereal beings and their magical dwellings."
        ]
    },
    {
        "tag": "enchanted_butterflies",
        "patterns": [
            "Encounters with enchanted butterflies",
            "The magic of butterfly wings",
            "The symbolism of butterflies in dreams"
        ],
        "responses": [
            "Enchanted butterflies are mythical creatures with wings that possess magical qualities or grant wishes.",
            "In reality, butterflies are delicate insects known for their beauty and symbolic significance."
        ]
    },
    {
        "tag": "time_warp_dance",
        "patterns": [
            "Dancing in a time warp",
            "The fusion of dance and time travel",
            "Time-traveling dance moves"
        ],
        "responses": [
            "Time warp dance is an imaginative concept where dance transcends time and allows people to experience different eras.",
            "Dance has evolved and adapted across cultures and historical periods."
        ]
    },
    {
        "tag": "fantasy_lands",
        "patterns": [
            "Tell me about mystical fantasy lands",
            "Exploring lands of wonder and enchantment",
            "Traveling to parallel dimensions"
        ],
        "responses": [
            "Fantasy lands are captivating worlds filled with magic, mythical creatures, and breathtaking landscapes.",
            "Books and movies have introduced audiences to imaginative realms beyond the boundaries of our reality."
        ]
    },
    {
        "tag": "enchanted_paintings",
        "patterns": [
            "Discovering enchanted paintings",
            "Art that comes to life",
            "Portals through magical artworks"
        ],
        "responses": [
            "Enchanted paintings are artistic wonders that blur the line between the painted and the real world.",
            "Art has the power to evoke emotions, ignite imagination, and inspire creativity."
        ]
    },
    {
        "tag": "spellbook_collection",
        "patterns": [
            "Collecting spellbooks from distant lands",
            "The allure of ancient tomes",
            "Studying spells and incantations"
        ],
        "responses": [
            "Spellbook collections are treasured possessions that hold knowledge of magical arts and wisdom.",
            "In reality, book collectors and enthusiasts value rare and ancient texts."
        ]
    },
    {
        "tag": "enchanted_sculptures",
        "patterns": [
            "Tell me about enchanted sculptures",
            "Statues with mystical powers",
            "Transformative art installations"
        ],
        "responses": [
            "Enchanted sculptures are artistic creations that possess magical properties or evoke a sense of wonder and awe.",
            "Sculptures are expressive forms of art that can evoke emotions, tell stories, and reflect cultural ideas."
        ]
    },
    {
        "tag": "wizarding_sports",
        "patterns": [
            "Playing wizarding sports",
            "Quidditch-like games in real life",
            "Magical athletes and their feats"
        ],
        "responses": [
            "Wizarding sports, like Quidditch from Harry Potter, showcase magical athleticism and exciting competitions.",
            "In the real world, sports and athletic competitions bring people together in a spirit of camaraderie and healthy competition."
        ]
    },
    {
        "tag": "enchanted_gems",
        "patterns": [
            "Unearthing enchanted gems",
            "Gemstones with mystical properties",
            "The power of gemstone magic"
        ],
        "responses": [
            "Enchanted gems are mythical gemstones that possess magical qualities, such as healing or granting wishes.",
            "Gemstones have long been treasured for their beauty, rarity, and symbolism."
        ]
    },
    {
        "tag": "timeless_wanderer",
        "patterns": [
            "The adventures of a timeless wanderer",
            "Roaming through eras and dimensions",
            "The wisdom of an eternal traveler"
        ],
        "responses": [
            "A timeless wanderer is a mythical figure, often depicted as an immortal or time-traveling being who observes the ebb and flow of history.",
            "Travelers in reality explore the world, experiencing diverse cultures and creating lasting memories."
        ]
    },
    {
        "tag": "enchanted_mirror_maze",
        "patterns": [
            "Getting lost in an enchanted mirror maze",
            "Reflections and illusions in the maze",
            "The challenge of finding the way out"
        ],
        "responses": [
            "An enchanted mirror maze is a bewitching labyrinth filled with reflections and optical illusions.",
            "Real-world mirror mazes are popular attractions that offer fun and disorienting experiences."
        ]
    },
    {
        "tag": "astral_artistry",
        "patterns": [
            "Exploring astral artistry",
            "Paintings that transcend dimensions",
            "Art that captures cosmic beauty"
        ],
        "responses": [
            "Astral artistry is an imaginative concept where art reflects celestial wonders and the mysteries of the universe.",
            "Artists use their creativity to portray their perceptions of the world and beyond."
        ]
    },
    {
        "tag": "enchanted_teapot",
        "patterns": [
            "Encountering an enchanted teapot",
            "Tea ceremonies in magical realms",
            "Teapot magic and rituals"
        ],
        "responses": [
            "An enchanted teapot is a fantastical vessel that brews magical tea with extraordinary properties.",
            "Tea ceremonies have cultural significance in various traditions and symbolize hospitality and connection."
        ]
    },
    {
        "tag": "timeless_ballroom",
        "patterns": [
            "Dancing in a timeless ballroom",
            "Elegant waltzes through the ages",
            "Time-traveling masquerade balls"
        ],
        "responses": [
            "A timeless ballroom is a mythical setting where dance transcends time, and participants waltz through different historical eras.",
            "Ballroom dancing is a graceful and expressive form of dance that has evolved over centuries."
        ]
    },
    {
        "tag": "enchanted_fog",
        "patterns": [
            "Encountering enchanted fog",
            "The mysteries within mystical mists",
            "Foggy journeys through magical lands"
        ],
        "responses": [
            "Enchanted fog is a magical phenomenon that cloaks landscapes in mystery and wonder.",
            "Fog is a natural meteorological condition caused by the condensation of water vapor in the air."
        ]
    },
    {
        "tag": "spellcasting_tournament",
        "patterns": [
            "Competing in a spellcasting tournament",
            "Magical duels and wizard battles",
            "The quest for the title of the grand spellcaster"
        ],
        "responses": [
            "Spellcasting tournaments are thrilling events where magic-users showcase their skills and compete for recognition and honor.",
            "Competitions and sports events celebrate talent, dedication, and teamwork."
        ]
    },
    {
        "tag": "enchanted_stardust",
        "patterns": [
            "Discovering enchanted stardust",
            "Cosmic particles with magical properties",
            "Wishes granted by stardust"
        ],
        "responses": [
            "Enchanted stardust is a mythical substance that is said to originate from the cosmos and hold magical properties.",
            "Stardust is a poetic term that refers to cosmic particles that form stars and planets."
        ]
    },
    {
        "tag": "fantastical_architecture",
        "patterns": [
            "Marveling at fantastical architecture",
            "Buildings that defy gravity and logic",
            "Designing dreamlike structures"
        ],
        "responses": [
            "Fantastical architecture is an artistic expression that pushes the boundaries of imagination, creating whimsical and surreal buildings.",
            "Architects and designers explore innovative ideas to shape the future of urban living."
        ]
    },
    {
        "tag": "enchanted_breeze",
        "patterns": [
            "Feeling the touch of an enchanted breeze",
            "The magic of gentle winds",
            "Whispers of mystical winds"
        ],
        "responses": [
            "An enchanted breeze is a mythical concept where gentle winds carry magic and blessings.",
            "The wind is a natural atmospheric phenomenon that influences weather patterns and is essential for the environment."
        ]
    },
    {
        "tag": "dragon_tamer",
        "patterns": [
            "Becoming a dragon tamer",
            "The art of befriending dragons",
            "Forming bonds with mythical beasts"
        ],
        "responses": [
            "Dragon tamers are legendary figures who form unique connections with dragons and earn their trust.",
            "Dragons symbolize power and wisdom in various cultures and mythologies."
        ]
    },
    {
        "tag": "enchanted_fruits",
        "patterns": [
            "Discovering enchanted fruits",
            "Fruits with mystical flavors and effects",
            "The magical properties of rare fruits"
        ],
        "responses": [
            "Enchanted fruits are mythical creations that possess extraordinary flavors or grant fantastical abilities.",
            "In reality, fruits are nutritious and delicious natural foods that offer a wide range of health benefits."
        ]
    },
    {
        "tag": "magical_mistakes",
        "patterns": [
            "Learning from magical mistakes",
            "Lessons in wizardry gone awry",
            "The humor and consequences of misfiring spells"
        ],
        "responses": [
            "Magical mistakes are comedic elements in fantasy stories where spells and enchantments don't go as planned.",
            "In real life, making mistakes is a part of learning and growing."
        ]
    },
    {
        "tag": "enchanted_rain",
        "patterns": [
            "Experiencing enchanted rain",
            "Rainfall that brings good fortune",
            "Dancing in the rain with magical properties"
        ],
        "responses": [
            "Enchanted rain is a fantastical concept where raindrops hold magical qualities, such as granting blessings or wishes.",
            "Rain is a natural weather phenomenon and an essential part of the water cycle."
        ]
    },
    {
        "tag": "timeless_tales",
        "patterns": [
            "Sharing timeless tales",
            "Storytelling that spans generations",
            "The magic of passing down stories"
        ],
        "responses": [
            "Timeless tales are enduring narratives that transcend time and continue to captivate audiences across generations.",
            "Storytelling is a powerful form of communication and cultural preservation."
        ]
    },
    {
        "tag": "enchanted_well",
        "patterns": [
            "Encountering an enchanted well",
            "Wishes granted by magical waters",
            "Seeking wisdom from the depths"
        ],
        "responses": [
            "An enchanted well is a mythical source of water that grants wishes or possesses healing properties.",
            "Wells have historical significance as vital sources of water for communities."
        ]
    },
    {
        "tag": "dimensional_doorway",
        "patterns": [
            "Opening a dimensional doorway",
            "Portals to parallel worlds",
            "Exploring alternate realities"
        ],
        "responses": [
            "A dimensional doorway is an imaginative concept that allows passage between different dimensions and realities.",
            "The idea of parallel universes and multiple dimensions is a popular theme in science fiction and theoretical physics."
        ]
    },
    {
        "tag": "enchanted_snow",
        "patterns": [
            "Witnessing enchanted snowfall",
            "The magic of snow in mythical realms",
            "The secrets of eternal snow"
        ],
        "responses": [
            "Enchanted snow is a mythical phenomenon where snowflakes possess magical properties or create enchanting winter landscapes.",
            "Snow is a meteorological event where frozen water crystals fall from the sky in colder climates."
        ]
    },
    {
        "tag": "spellbinding_cuisine",
        "patterns": [
            "Savoring spellbinding cuisine",
            "Food that delights the senses and soul",
            "Recipes from ancient spellbooks"
        ],
        "responses": [
            "Spellbinding cuisine is an imaginative concept where food is prepared with magical ingredients and flavors.",
            "Cooking and culinary arts offer diverse tastes and cultural experiences."
        ]
    },
    {
        "tag": "enchanted_lullabies",
        "patterns": [
            "Listening to enchanted lullabies",
            "Songs that soothe and enchant",
            "The magic of bedtime melodies"
        ],
        "responses": [
            "Enchanted lullabies are mythical songs that have a soothing and magical effect, lulling listeners into a peaceful slumber.",
            "Lullabies are cherished melodies that parents sing to their children to comfort and calm them."
        ]
    },
    {
        "tag": "time_travelers_journal",
        "patterns": [
            "Reading a time traveler's journal",
            "Chronicles of historical adventures",
            "Insights into the past and future"
        ],
        "responses": [
            "A time traveler's journal is a fictional account documenting journeys through different historical periods and possibly into the future.",
            "Historical records, memoirs, and diaries provide valuable insights into the past."
        ]
    },
    {
        "tag": "enchanted_stables",
        "patterns": [
            "Visiting enchanted stables",
            "Magical creatures and their stables",
            "Horses with mythical qualities"
        ],
        "responses": [
            "Enchanted stables are fantastical settings where mythical creatures, such as winged horses, are cared for and nurtured.",
            "Stables in reality house domesticated animals, providing shelter and care for their well-being."
        ]
    },
    {
        "tag": "wizarding_cuisine",
        "patterns": [
            "Exploring wizarding cuisine",
            "Magical ingredients and recipes",
            "Feasting in the grand wizarding halls"
        ],
        "responses": [
            "Wizarding cuisine is a fictional culinary world where food is prepared with magical flair and sometimes holds mystical properties.",
            "Culinary diversity and creativity are celebrated across different cultures."
        ]
    },
    {
        "tag": "enchanted_riddles",
        "patterns": [
            "Solving enchanted riddles",
            "Riddles that guard hidden treasures",
            "Wisdom and challenges in mythical riddles"
        ],
        "responses": [
            "Enchanted riddles are puzzles that challenge the mind and are often associated with quests and adventures in folklore and fantasy tales.",
            "Riddles have been used for entertainment and intellectual stimulation throughout history."
        ]
    },
    {
        "tag": "timeless_relics",
        "patterns": [
            "Discovering timeless relics",
            "Artifacts from lost civilizations",
            "The allure of ancient mysteries"
        ],
        "responses": [
            "Timeless relics are ancient artifacts and objects that hold historical and cultural significance, preserving the stories of our ancestors.",
            "Archaeologists and historians continue to unearth relics that shed light on human history and civilization."
        ]
    },
    {
      "tag": "creator",
      "patterns": ["Who created you?", "Who is your developer?", "Who made you?"],
      "responses": ["I was created by Lokesh Rowthula."],
      "context_set": ""
    }
]

In [24]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Processing Data and Synonym Augmentation for Intent Classification

In [26]:
stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


# Directly iterate through the 'intents' list
for intent in intents:  # Removed ['intents']
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

838 documents
274 classes ['about', 'age', 'alien_invasion', 'alien_languages', 'arcane_alchemy', 'art', 'art_and_crafts', 'artificial_intelligence', 'artificial_superintelligence', 'astral_artistry', 'astral_projection', 'augmented_reality', 'baking', 'biomedical_engineering', 'book_of_spells', 'books', 'brain_computer_interface', 'budget', 'budgeting_tips', 'butterfly_language', 'car_maintenance', 'career', 'charity_donations', 'chocolate_obsession', 'cloud_castle_ruler', 'cloud_castles', 'coding', 'coding_languages', 'comedy_tonic', 'compliment', 'composting', 'cooking', 'cooking_for_special_occasions', 'cooking_tips', 'creative_writing', 'creativity', 'creator', 'credit_score', 'cuisine', 'cybersecurity', 'daily_meditation', 'dancing', 'data_privacy', 'depression', 'developing_creativity_in_children', 'digital_organization', 'dimensional_doorway', 'dragon_riding', 'dragon_tamer', 'dream_world', 'eco-friendly_transportation', 'education', 'effective_communication_in_relationships', 

In [27]:
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)


def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences


# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        # Ensure augmented_bag has the same length as the original bag
        augmented_bag = [0] * len(words)  # Initialize with zeros
        for word_index, word in enumerate(words):
            if augmented_sentence.find(word) >= 0:
                augmented_bag[word_index] = 1
        augmented_data.append([augmented_bag, output_row])

# Convert training and augmented_data to NumPy arrays before concatenating
training_np = np.array(training, dtype=object)
augmented_data_np = np.array(augmented_data, dtype=object)

# Concatenate the NumPy arrays
combined_data = np.concatenate((training_np, augmented_data_np), axis=0)
random.shuffle(combined_data)

# Splitting the Dataset

In [28]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

# Training The Model

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [30]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.0206, Training Accuracy: 0.0885
Epoch [1/50], Testing Loss: 0.0158, Testing Accuracy: 0.2680
Epoch [2/50], Training Loss: 0.0116, Training Accuracy: 0.5073
Epoch [2/50], Testing Loss: 0.0084, Testing Accuracy: 0.6614
Epoch [3/50], Training Loss: 0.0067, Training Accuracy: 0.7382
Epoch [3/50], Testing Loss: 0.0055, Testing Accuracy: 0.7876
Epoch [4/50], Training Loss: 0.0047, Training Accuracy: 0.8219
Epoch [4/50], Testing Loss: 0.0041, Testing Accuracy: 0.8454
Epoch [5/50], Training Loss: 0.0036, Training Accuracy: 0.8661
Epoch [5/50], Testing Loss: 0.0033, Testing Accuracy: 0.8769
Epoch [6/50], Training Loss: 0.0029, Training Accuracy: 0.8919
Epoch [6/50], Testing Loss: 0.0028, Testing Accuracy: 0.8981
Epoch [7/50], Training Loss: 0.0024, Training Accuracy: 0.9100
Epoch [7/50], Testing Loss: 0.0024, Testing Accuracy: 0.9131
Epoch [8/50], Training Loss: 0.0021, Training Accuracy: 0.9234
Epoch [8/50], Testing Loss: 0.0021, Testing Accuracy: 0.9218
Epoch [9

# Model Inference

In [31]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    #print(f"Original Sentence: {sentence}")  # Commented out
    sentence_words = preprocess_sentence(sentence, words)
    #print(f"Preprocessed Words: {sentence_words}") # Commented out
    if len(sentence_words) == 0:
        return "I'm sorry, but I don't understand. Can you please rephrase or provide more information?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    #print(f"Confidence: {confidence}") # Commented out
    predicted_tag = classes[predicted_class.item()]
    #print(f"Predicted Tag: {predicted_tag}") # Commented out

    if confidence > 0.3:
        for intent in intents:
            if intent['tag'] == predicted_tag:
                response = intent['responses'][0]
                #print(f"Selected Response: {response}") # Commented out
                return response

    return "I'm sorry, but I'm not sure how to respond to that."

In [37]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input, model, words, classes)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
> quit


In [35]:
import textwrap

test_cases = [
    "Hello",
    "Good Morning",
    "What are you",
    "Who Created You",
    "Thats not right",
    "What are the class sizes like?",
    "Does your major require an independent study?",
    "Are professors available for research with students?",
    "Are there study abroad programs available?",
    "What's it like to be a first-year student here?",
    "Can students spend a semester abroad?",
    "How can I get involved in research at your college?",
    "Tell me about the extracurricular activities on campus.",
    "What resources are available for career counseling and job placement?",
    "Does the school offer any scholarships or financial aid for international students?",
    "How does the campus support students with disabilities?",
    "Are there any on-campus housing options for upperclassmen?",
    "What opportunities are there for students interested in community service and volunteering?",
    "Is there a mentoring program for freshmen to help with their transition to college life?",
    "What type of on-campus dining options are available?",
    "Tell me about the campus safety measures and security services.",
    "What type of technology and computer facilities are accessible to students?",
    "Are there any notable guest speakers or events that regularly occur on campus?",
    "How does the college foster a sense of community and inclusivity among students?",
    "What is the average student-to-faculty ratio at the university?",
    "Can students participate in sports or fitness activities even if they are not part of a varsity team?",
    "Are there any unique academic programs or centers of excellence at the college?",
    "What type of student organizations are available for students to join?",
    "Tell me about the college's commitment to sustainability and environmental initiatives.",
    "How does the college handle academic support and tutoring for students who need extra help?",
    "Are there any opportunities for students to showcase their artistic talents, such as theater performances or art exhibitions?",
    "Does the college have any partnerships with local businesses or organizations for internships and experiential learning?",
    "What are the on-campus recreational facilities, such as fitness centers or sports fields?",
    "What are the admission requirements for international students?",
    "Can you tell me about the campus facilities, such as the library and student center?",
    "What types of student support services are available, such as counseling and health services?",
    "How diverse is the student body on campus?",
    "What are the top academic programs offered at the college?",
    "Does the college have any special programs for first-generation college students?",
    "What is the college's policy on AP credits and advanced placement?",
    "Are there opportunities for undergraduate students to participate in research with faculty?",
    "What kind of financial aid packages are available for low-income students?",
    "Can you provide information on the college's alumni network and career connections?",
    "Tell me about the college's approach to experiential learning and internships.",
    "What are the campus housing options for students who prefer a single room?",
    "Does the college offer any honors or honors programs for high-achieving students?",
    "What are the popular social events and traditions on campus?",
    "How does the college support students in finding part-time jobs while studying?",
    "Can you tell me more about the college's study abroad partnerships and locations?",
    "What is the college's policy on transfer credits?",
    "Are there any special resources or clubs for students interested in entrepreneurship?",
    "Tell me about the college's sustainability initiatives and green campus efforts.",
    "How accessible are professors for office hours and academic support?",
    "What are the research opportunities available for undergraduate students?",
    "Can you provide information on the college's campus safety statistics and measures?",
    "How does the college celebrate diversity and promote inclusivity on campus?",
    "What are the available options for transportation to and from campus?",
    "Tell me about the college's alumni mentoring programs and career networking events.",
    "Are there any leadership development programs or student government opportunities?",
    "How does the college handle academic advising and course registration for new students?",
    "What are the college's facilities for performing arts and music?",
    "Can you provide information on the college's community service requirements?",
    "Tell me about the college's approach to experiential learning and internships.",
    "What are the campus housing options for students who prefer a single room?",
    "Does the college offer any honors or honors programs for high-achieving students?",
    "What are the popular social events and traditions on campus?",
    "How does the college support students in finding part-time jobs while studying?",
    "Can you tell me more about the college's study abroad partnerships and locations?",
    "What is the college's policy on transfer credits?",
    "Are there any special resources or clubs for students interested in entrepreneurship?",
    "Tell me about the college's sustainability initiatives and green campus efforts.",
    "How accessible are professors for office hours and academic support?",
    "What are the research opportunities available for undergraduate students?",
    "Can you provide information on the college's campus safety statistics and measures?",
    "How does the college celebrate diversity and promote inclusivity on campus?",
    "What are the available options for transportation to and from campus?",
    "Tell me about the college's alumni mentoring programs and career networking events.",
    "Are there any leadership development programs or student government opportunities?",
    "How does the college handle academic advising and course registration for new students?",
    "What are the college's facilities for performing arts and music?",
    "Can you provide information on the college's community service requirements?",
]
print('Testing Cases starting..........\n')

unseen = []
for i in test_cases:
  print(i)
  print('\n')
  text = generate_response(i.lower(),model, words, classes)
  print(textwrap.fill(text, width=70))
  print('-------------------------------------')
  if text == "I'm sorry, but I'm not sure how to respond to that.":
    unseen.append(i)

Testing Cases starting..........

Hello


Hi there
-------------------------------------
Good Morning


Goodbye
-------------------------------------
What are you


I am a chatbot
-------------------------------------
Who Created You


I was created by Lokesh Rowthula.
-------------------------------------
Thats not right


I'm sorry, but I'm not sure how to respond to that.
-------------------------------------
What are the class sizes like?


Magic schools, like the ones depicted in fantasy books and movies, are
fictional settings for learning spells and wizardry.
-------------------------------------
Does your major require an independent study?


I'm sorry, but I'm not sure how to respond to that.
-------------------------------------
Are professors available for research with students?


Science is the systematic study of the natural world, aiming to
understand phenomena and make discoveries.
-------------------------------------
Are there study abroad programs available?


Fairy 

In [36]:
unseen

['Thats not right',
 'Does your major require an independent study?',
 'Tell me about the extracurricular activities on campus.',
 'What type of on-campus dining options are available?',
 'What kind of financial aid packages are available for low-income students?']